In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

!pip install kaggle
!pip install -q ultralytics pycocotools easyocr opencv-python pillow matplotlib tqdm scikit-image
!pip install rembg
!pip install onnxruntime onnxruntime-gpu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 7s (1,428 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126718 files and dire

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ['KAGGLE_USERNAME'] = 'starsin'
os.environ['KAGGLE_KEY'] = 'ff559a02afd29d67f19bc4001d5a69f7'

api = KaggleApi()
api.authenticate()


# 2. 대회 데이터 다운로드
competition_name = "ai05-level1-project"
api.competition_download_files(competition_name, path="./data")

In [ ]:
import zipfile

zip_path = "./data/ai05-level1-project.zip"
extract_dir = "./data/ai05-level1-project"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

In [ ]:
# =========================================================
# 0) 설치 및 공통 임포트
# =========================================================
# !pip install -q ultralytics==8.3.58 pycocotools easyocr opencv-python pillow matplotlib tqdm scikit-image

import os, json, shutil, random
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
import cv2

from pycocotools.coco import COCO
from ultralytics import YOLO

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# =========================================================
# 1) 경로 및 기본 상수 설정
# =========================================================
DATA_ROOT = "./data/ai05-level1-project"
IMG_DIR   = f"{DATA_ROOT}/train_images"
ANN_DIR   = f"{DATA_ROOT}/train_annotations"
WORK_DIR  = "./work_yolov8"
YOLO_DIR  = f"{WORK_DIR}/yolo_dataset"
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(YOLO_DIR, exist_ok=True)

ALL_JSON   = f"{WORK_DIR}/train_all_robust.json"
TRAIN_JSON = f"{WORK_DIR}/train.json"
VAL_JSON   = f"{WORK_DIR}/val.json"
TEST_DIR   = os.path.join(DATA_ROOT, "test_images")

# ✅ 전 구간 이미지 해상도 640 고정
IMGSZ_SWEEP = 640
IMGSZ_FINAL = 640
print(f"✅ IMGSZ_SWEEP={IMGSZ_SWEEP}, IMGSZ_FINAL={IMGSZ_FINAL}")

random.seed(42)
np.random.seed(42)

✅ IMGSZ_SWEEP=640, IMGSZ_FINAL=640


In [ ]:
# =========================================================
# 2) 유틸 함수
# =========================================================
def norm_name(s:str)->str:
    return s.replace("\\","/").split("/")[-1].strip()

def file_exists_in_train_images(file_name:str)->bool:
    return os.path.exists(os.path.join(IMG_DIR, file_name))

def get_img_size_from_disk(file_name:str):
    p = os.path.join(IMG_DIR, file_name)
    with Image.open(p) as im:
        return im.size

def bbox_valid_clip(x,y,w,h,W,H):
    x2 = max(0.0, min(x+w, W))
    y2 = max(0.0, min(y+h, H))
    x1 = max(0.0, min(x,   W))
    y1 = max(0.0, min(y,   H))
    w2 = x2 - x1
    h2 = y2 - y1
    if w2 <= 1e-6 or h2 <= 1e-6:
        return None
    return [float(x1), float(y1), float(w2), float(h2)]

def dedup_key(new_img_id, cat_id, bbox):
    x,y,w,h = [round(float(v),2) for v in bbox]
    return (int(new_img_id), int(cat_id), x, y, w, h)

def hash_cat(c):
    return (str(c.get("name","")), str(c.get("supercategory","pill")))

In [ ]:
# =========================================================
# 4) COCO JSON 병합
# =========================================================
merged = {"images": [], "annotations": [], "categories": []}
cat_by_id, cat_hash_by_id = {}, {}
file_to_new_img_id, image_meta = {}, {}
new_img_id, ann_id = 1, 1
seen_ann_keys = set()
cnt = defaultdict(int)

for folder in sorted(os.listdir(ANN_DIR)):
    if not folder.endswith("_json"):
        continue
    cnt["folders_scanned"] += 1
    for dp, _, files in os.walk(os.path.join(ANN_DIR, folder)):
        for f in files:
            if not f.lower().endswith(".json"):
                continue
            cnt["json_scanned"] += 1
            jp = os.path.join(dp, f)
            try:
                j = json.load(open(jp))
            except Exception:
                continue
            for c in j.get("categories", []):
                cid = int(c["id"])
                h = hash_cat(c)
                if cid not in cat_by_id:
                    cat_by_id[cid] = {"id": cid, "name": c.get("name",""), "supercategory": c.get("supercategory","pill")}
                    cat_hash_by_id[cid] = h
            for im in j.get("images", []):
                file_name = norm_name(im.get("file_name",""))
                if not file_exists_in_train_images(file_name): continue
                if file_name not in file_to_new_img_id:
                    W,H = get_img_size_from_disk(file_name)
                    file_to_new_img_id[file_name] = new_img_id
                    image_meta[new_img_id] = {"id": new_img_id, "file_name": file_name, "width": W, "height": H}
                    new_img_id += 1
            for a in j.get("annotations", []):
                src_im = next((im for im in j.get("images", []) if im["id"] == a.get("image_id")), None)
                if not src_im: continue
                file_name = norm_name(src_im.get("file_name",""))
                if not file_name or file_name not in file_to_new_img_id: continue
                W,H = get_img_size_from_disk(file_name)
                fixed = bbox_valid_clip(*a["bbox"], W, H)
                if not fixed: continue
                k = dedup_key(file_to_new_img_id[file_name], a["category_id"], fixed)
                if k in seen_ann_keys: continue
                seen_ann_keys.add(k)
                merged["annotations"].append({
                    "id": ann_id, "image_id": file_to_new_img_id[file_name],
                    "category_id": a["category_id"], "bbox": fixed,
                    "area": float(fixed[2]*fixed[3]), "iscrowd": int(a.get("iscrowd", 0))
                })
                ann_id += 1

merged["images"] = [image_meta[i] for i in sorted(image_meta)]
merged["categories"] = [cat_by_id[c] for c in sorted(cat_by_id)]
json.dump(merged, open(ALL_JSON, "w"), indent=2, ensure_ascii=False)
print("✅ 병합 완료:", len(merged["images"]), "imgs", len(merged["annotations"]), "anns")

✅ 병합 완료: 1489 imgs 4524 anns


### 이전 데이터 학습

In [ ]:
# =========================================================
# SSIM 최적화 버전: 해시로 후보 축소 → 상위 K개만 SSIM
#  - OpenCV PHash(aHash보다 견고)로 1차 후보군 생성
#  - 해시 prefix 버킷(상위 N비트)으로 비교 수 급감
#  - 후보 중 해밍거리 상위 K개만 SSIM 정밀 체크
#  - multiprocessing으로 병렬
# =========================================================
import os, cv2, math, itertools, multiprocessing as mp
import numpy as np
from pathlib import Path
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim

TEST_DIR  = "./data/ai05-level1-project/test_images"
TRAIN_DIR = "./data/ai05-level1-project/train_images"

# ---- 파라미터 ----
RESIZE_WH   = 256      # 해시/SSIM 계산용 리사이즈 (작을수록 빠름)
PREFIX_BITS = 12       # 해시 prefix 버킷 길이(커질수록 비교 수 ↓, 너무 크면 누락↑)
HAM_RAD     = 6        # 1차 후보: 해밍거리 허용 반경
TOPK_SSIM   = 5        # SSIM은 후보 중 해밍거리 가까운 상위 K개만
SSIM_THR    = 0.99     # SSIM 유사 판정 임계치
N_WORKERS   = max(1, mp.cpu_count() - 1)

# ---- 유틸 ----
def imread_gray_resize(p, wh=RESIZE_WH):
    img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
    if img is None:
        return None
    if img.shape[0] != wh or img.shape[1] != wh:
        img = cv2.resize(img, (wh, wh), interpolation=cv2.INTER_AREA)
    return img

def phash_64(img):
    # OpenCV PHash는 8x8=64bit로 반환됨
    hasher = cv2.img_hash.PHash_create()
    h = hasher.compute(img).flatten()  # 8x8 uint8
    # OpenCV는 0/255로 채움 → bit로 변환
    bits = 0
    for i, v in enumerate(h):
        if v > 0:
            bits |= (1 << i)
    return bits

def hamming64(a, b):
    return (a ^ b).bit_count()

def hash_prefix(h, prefix_bits=PREFIX_BITS):
    # 상위 prefix_bits만 남기고 나머지 마스킹
    if prefix_bits <= 0:
        return 0
    shift = 64 - prefix_bits
    return (h >> shift) & ((1 << prefix_bits) - 1)

# ---- 1) Train PHash 인덱스 구성 (prefix 버킷) ----
train_files = [os.path.join(TRAIN_DIR, f) for f in os.listdir(TRAIN_DIR)
               if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]

bucket = {}  # prefix -> list of (hash, path)
print("Hash-train (PHash):")
for p in tqdm(train_files):
    g = imread_gray_resize(p)
    if g is None:
        continue
    h = phash_64(g)
    pref = hash_prefix(h)
    bucket.setdefault(pref, []).append((h, p))

# ---- 2) Test에 대해 후보 축소 + 상위K SSIM ----
test_files = [os.path.join(TEST_DIR, f) for f in os.listdir(TEST_DIR)
              if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]

def process_one(test_path):
    ti = imread_gray_resize(test_path)
    if ti is None:
        return None
    th = phash_64(ti)
    pref = hash_prefix(th)

    # 동일 prefix 버킷만 비교 → 해밍 필터로 1차 후보 축소
    cand = bucket.get(pref, [])
    if not cand:
        return None

    # 해밍 거리 계산 후 반경 필터
    ham_list = [(hamming64(th, ch), cp) for (ch, cp) in cand]
    ham_list = [x for x in ham_list if x[0] <= HAM_RAD]
    if not ham_list:
        return None

    # 해밍 가까운 순으로 TOPK_SSIM만 SSIM
    ham_list.sort(key=lambda x: x[0])
    pick = ham_list[:TOPK_SSIM]

    best = (0.0, None)  # (ssim, path)
    for _, cand_path in pick:
        ci = imread_gray_resize(cand_path)
        if ci is None:
            continue
        s = ssim(ti, ci)
        if s > best[0]:
            best = (s, cand_path)
        if s >= SSIM_THR:  # 충분히 동일하면 조기종료
            break

    if best[1] is None:
        return None
    return (test_path, best[1], best[0])

print("SSIM-check (optimized):")
with mp.Pool(N_WORKERS) as pool:
    out = list(tqdm(pool.imap_unordered(process_one, test_files), total=len(test_files)))

# 결과 정리 (None 제외)
leaks = [x for x in out if x is not None and x[2] >= SSIM_THR]
print(f"⚠️ 누수 의심(SSIM ≥ {SSIM_THR}) : {len(leaks)}건")
for a,b,s in leaks[:10]:
    print(" -", Path(a).name, "<->", Path(b).name, f"SSIM={s:.4f}")

# CSV 저장
import pandas as pd
os.makedirs("./work_yolov8", exist_ok=True)
csv_path = "./work_yolov8/leak_ssim_optimized.csv"
rows = [{"test_path": a, "train_path": b, "ssim": float(s)} for a,b,s in leaks]
pd.DataFrame(rows).to_csv(csv_path, index=False, encoding="utf-8-sig")
print("📄 saved:", csv_path)

Hash-train (PHash):


100%|██████████| 1489/1489 [00:55<00:00, 26.65it/s]

SSIM-check (optimized):



100%|██████████| 843/843 [00:25<00:00, 32.68it/s]


⚠️ 누수 의심(SSIM ≥ 0.99) : 11건
 - 9.png <-> K-003483-020238-022347-027653_0_2_0_2_75_000_200.png SSIM=1.0000
 - 1010.png <-> K-003483-016262-022347-025469_0_2_0_2_90_000_200.png SSIM=1.0000
 - 465.png <-> K-003483-020238-027653-028763_0_2_0_2_90_000_200.png SSIM=1.0000
 - 1329.png <-> K-001900-016548-029451-044199_0_2_0_2_75_000_200.png SSIM=1.0000
 - 1039.png <-> K-003483-016262-025469-028763_0_2_0_2_90_000_200.png SSIM=1.0000
 - 1141.png <-> K-002483-012081-023223-025438_0_2_0_2_70_000_200.png SSIM=1.0000
 - 621.png <-> K-002483-005094-022362-023223_0_2_0_2_90_000_200.png SSIM=1.0000
 - 128.png <-> K-002483-005094-006192-022362_0_2_0_2_90_000_200.png SSIM=1.0000
 - 395.png <-> K-003483-020238-025469-028763_0_2_0_2_90_000_200.png SSIM=1.0000
 - 396.png <-> K-003483-020238-025469-028763_0_2_0_2_75_000_200.png SSIM=1.0000
📄 saved: ./work_yolov8/leak_ssim_optimized.csv


In [ ]:
# ===== 누수 재확인: 범위 더 넓혀서 스캔(놓친 케이스 방지) =====
# - prefix_bits ↓, ham 반경 ↑, SSIM 후보 TOPK ↑
# - 결과 CSV: ./work_yolov8/leak_ssim_optimized_wide.csv

import os, cv2, multiprocessing as mp
import numpy as np
from tqdm import tqdm
from pathlib import Path
from skimage.metrics import structural_similarity as ssim

TEST_DIR  = "./data/ai05-level1-project/test_images"
TRAIN_DIR = "./data/ai05-level1-project/train_images"
WORK_DIR  = "./work_yolov8"
os.makedirs(WORK_DIR, exist_ok=True)

RESIZE_WH   = 256
PREFIX_BITS = 10   # ↓ (버킷 넓힘)
HAM_RAD     = 10   # ↑ (더 많은 후보 허용)
TOPK_SSIM   = 8    # ↑
SSIM_THR    = 0.99
N_WORKERS   = max(1, mp.cpu_count() - 1)

def imread_gray_resize(p, wh=RESIZE_WH):
    img = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
    if img is None: return None
    if img.shape[:2] != (wh, wh):
        img = cv2.resize(img, (wh, wh), interpolation=cv2.INTER_AREA)
    return img

def phash_64(img):
    hasher = cv2.img_hash.PHash_create()
    h = hasher.compute(img).flatten()
    bits = 0
    for i, v in enumerate(h):
        if v > 0:
            bits |= (1 << i)
    return bits

def hamming64(a, b): return (a ^ b).bit_count()
def hash_prefix(h, prefix_bits=PREFIX_BITS):
    shift = 64 - prefix_bits
    return (h >> shift) & ((1 << prefix_bits) - 1)

train_files = [os.path.join(TRAIN_DIR, f) for f in os.listdir(TRAIN_DIR)
               if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]
test_files  = [os.path.join(TEST_DIR, f) for f in os.listdir(TEST_DIR)
               if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]

# 1) Train 해시 버킷
bucket = {}
print("Hash-train (PHash, wide):")
for p in tqdm(train_files):
    g = imread_gray_resize(p)
    if g is None: continue
    h = phash_64(g)
    pref = hash_prefix(h)
    bucket.setdefault(pref, []).append((h, p))

def process_one(test_path):
    ti = imread_gray_resize(test_path)
    if ti is None: return None
    th = phash_64(ti)
    pref = hash_prefix(th)
    cand = bucket.get(pref, [])
    if not cand: return None

    ham = [(hamming64(th, ch), cp) for (ch, cp) in cand if hamming64(th, ch) <= HAM_RAD]
    if not ham: return None
    ham.sort(key=lambda x: x[0])
    pick = ham[:TOPK_SSIM]

    best = (0.0, None)
    for _, cp in pick:
        ci = imread_gray_resize(cp)
        if ci is None: continue
        s = ssim(ti, ci)
        if s > best[0]:
            best = (s, cp)
        if s >= SSIM_THR:
            break
    return (test_path, best[1], best[0]) if best[1] is not None else None

print("SSIM-check (wide):")
with mp.Pool(N_WORKERS) as pool:
    out = list(tqdm(pool.imap_unordered(process_one, test_files), total=len(test_files)))

leaks = [x for x in out if x is not None and x[2] >= SSIM_THR]
print(f"⚠️ 누수 의심(SSIM ≥ {SSIM_THR}) : {len(leaks)}건")
for a,b,s in leaks[:15]:
    print(" -", Path(a).name, "<->", Path(b).name, f"SSIM={s:.4f}")

import pandas as pd
csv_wide = os.path.join(WORK_DIR, "leak_ssim_optimized_wide.csv")
pd.DataFrame([{"test_path":a, "train_path":b, "ssim":float(s)} for a,b,s in leaks])\
  .to_csv(csv_wide, index=False, encoding="utf-8-sig")
print("📄 saved:", csv_wide)

Hash-train (PHash, wide):


100%|██████████| 1489/1489 [00:52<00:00, 28.57it/s]

SSIM-check (wide):



100%|██████████| 843/843 [00:38<00:00, 22.06it/s]


⚠️ 누수 의심(SSIM ≥ 0.99) : 14건
 - 9.png <-> K-003483-020238-022347-027653_0_2_0_2_75_000_200.png SSIM=1.0000
 - 465.png <-> K-003483-020238-027653-028763_0_2_0_2_90_000_200.png SSIM=1.0000
 - 1010.png <-> K-003483-016262-022347-025469_0_2_0_2_90_000_200.png SSIM=1.0000
 - 1329.png <-> K-001900-016548-029451-044199_0_2_0_2_75_000_200.png SSIM=1.0000
 - 1039.png <-> K-003483-016262-025469-028763_0_2_0_2_90_000_200.png SSIM=1.0000
 - 1141.png <-> K-002483-012081-023223-025438_0_2_0_2_70_000_200.png SSIM=1.0000
 - 621.png <-> K-002483-005094-022362-023223_0_2_0_2_90_000_200.png SSIM=1.0000
 - 128.png <-> K-002483-005094-006192-022362_0_2_0_2_90_000_200.png SSIM=1.0000
 - 1367.png <-> K-003483-022347-027653-029667_0_2_0_2_75_000_200.png SSIM=1.0000
 - 395.png <-> K-003483-020238-025469-028763_0_2_0_2_90_000_200.png SSIM=1.0000
 - 962.png <-> K-001900-016551-019607-029451_0_2_0_2_75_000_200.png SSIM=1.0000
 - 1009.png <-> K-003483-016262-022347-025469_0_2_0_2_75_000_200.png SSIM=1.0000
 - 396.p

### 신규 블랙시트 생성(저장완료)

In [ ]:
# =========================================================
# 강화 누수 점검: 파일명 stem, Exact Hash(MD5), Perceptual Hash(aHash), SSIM
# - TEST_DIR/ TRAIN_DIR 자동 탐색 개선
# - 결과 CSV 3종 저장
# =========================================================
import os, cv2, random, hashlib, numpy as np, pandas as pd
from pathlib import Path
from tqdm import tqdm
from skimage.metrics import structural_similarity as ssim
from PIL import Image

# ---------- 경로 후보 세팅 ----------
EXTS = (".jpg",".jpeg",".png",".bmp",".webp")

def has_images(d):
    return os.path.isdir(d) and any(f.lower().endswith(EXTS) for f in os.listdir(d))

DATA_ROOT = globals().get("DATA_ROOT", "./data/ai05-level1-project")
WORK_DIR  = globals().get("WORK_DIR", "./work_yolov8")
YOLO_DIR  = globals().get("YOLO_DIR", "./work_yolov8/yolo_dataset")
IMG_DIR   = globals().get("IMG_DIR", os.path.join(DATA_ROOT, "images"))
TEST_DIR0 = globals().get("TEST_DIR", None)

test_candidates = [
    TEST_DIR0,
    os.path.join(DATA_ROOT, "test_images"),
    os.path.join(DATA_ROOT, "test"),
    os.path.join(YOLO_DIR, "images", "test"),
    os.path.join(YOLO_DIR, "images", "val"),   # 차선: 검증셋으로라도 누수 의심 확인
    os.path.join(DATA_ROOT, "val_images"),
]
train_candidates = [
    os.path.join(YOLO_DIR, "images", "train"),
    os.path.join(DATA_ROOT, "train_images"),
    IMG_DIR,  # COCO 이미지 루트
]

def pick_first(cands):
    for p in cands:
        if p and has_images(p):
            return p
    return None

TEST_DIR  = pick_first(test_candidates)
TRAIN_DIR = pick_first(train_candidates)

print("TEST_DIR :", TEST_DIR or "(없음)")
print("TRAIN_DIR:", TRAIN_DIR or "(없음)")
if not TEST_DIR or not TRAIN_DIR:
    raise RuntimeError("이미지 폴더를 찾지 못했습니다. 경로를 확인하세요.")

def list_images(d):
    return [os.path.join(d, f) for f in os.listdir(d) if f.lower().endswith(EXTS)]

test_files  = sorted(list_images(TEST_DIR))
train_files = sorted(list_images(TRAIN_DIR))
print(f"#test={len(test_files)}  #train={len(train_files)}")

os.makedirs(WORK_DIR, exist_ok=True)

# ---------- 1) 파일명 stem 중복 ----------
def stem(p): return Path(p).stem
train_stems = set(stem(p) for p in train_files)
name_leaks = [(tf, stem(tf)) for tf in test_files if stem(tf) in train_stems]
df_name = pd.DataFrame(name_leaks, columns=["test_path","stem"])
out_name = os.path.join(WORK_DIR, "leak_name_stem_overlap.csv")
df_name.to_csv(out_name, index=False, encoding="utf-8-sig")
print(f"🔎 파일명 stem 중복: {len(df_name)}건 → {out_name}")

# ---------- 2) Exact Hash (MD5) 완전 동일 ----------
def md5_of_image(path, resize_to=None):
    try:
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        if img is None:
            return None
        if resize_to:
            img = cv2.resize(img, resize_to, interpolation=cv2.INTER_AREA)
        return hashlib.md5(img.tobytes()).hexdigest()
    except Exception:
        return None

# 해시 계산(속도 위해 샘플링 가능)
MAX_TRAIN_HASH = min(5000, len(train_files))
MAX_TEST_HASH  = min(2000, len(test_files))
train_files_h  = random.sample(train_files, MAX_TRAIN_HASH) if len(train_files) > MAX_TRAIN_HASH else train_files
test_files_h   = random.sample(test_files,  MAX_TEST_HASH)  if len(test_files)  > MAX_TEST_HASH  else test_files

# 동일 크기 리사이즈(완전 동일성만 보려면 resize_to=None 도 가능)
RESIZE_TO = (256, 256)

train_hash_map = {}
for p in tqdm(train_files_h, desc="Hash-train"):
    h = md5_of_image(p, resize_to=RESIZE_TO)
    if h: train_hash_map.setdefault(h, []).append(p)

exact_matches = []
for p in tqdm(test_files_h, desc="Hash-test"):
    h = md5_of_image(p, resize_to=RESIZE_TO)
    if not h: continue
    if h in train_hash_map:
        for tp in train_hash_map[h]:
            exact_matches.append((p, tp, h))
df_hash = pd.DataFrame(exact_matches, columns=["test_path","train_path","md5"])
out_hash = os.path.join(WORK_DIR, "leak_exact_hash_md5.csv")
df_hash.to_csv(out_hash, index=False, encoding="utf-8-sig")
print(f"🔎 MD5(리사이즈 {RESIZE_TO}) 완전 동일: {len(df_hash)}건 → {out_hash}")

# ---------- 3) Perceptual Hash (aHash) 근접 ----------
# PIL의 평균 해시 구현 (8x8 -> 64bit), 허용 해밍 거리 설정
def ahash(path, size=8):
    try:
        im = Image.open(path).convert("L").resize((size, size), Image.BILINEAR)
        px = np.asarray(im, dtype=np.float32)
        avg = px.mean()
        bits = (px > avg).flatten().astype(np.uint8)
        # 64비트 정수 문자열로 저장
        return ''.join('1' if b else '0' for b in bits)
    except Exception:
        return None

def hamming(a, b):
    return sum(ch1 != ch2 for ch1, ch2 in zip(a, b)) if (a and b and len(a)==len(b)) else 64

MAX_TRAIN_AHASH = min(5000, len(train_files))
MAX_TEST_AHASH  = min(1000, len(test_files))
train_files_a   = random.sample(train_files, MAX_TRAIN_AHASH) if len(train_files) > MAX_TRAIN_AHASH else train_files
test_files_a    = random.sample(test_files,  MAX_TEST_AHASH)  if len(test_files)  > MAX_TEST_AHASH  else test_files

train_ahashes = [(p, ahash(p)) for p in tqdm(train_files_a, desc="aHash-train")]
test_ahashes  = [(p, ahash(p)) for p in tqdm(test_files_a,  desc="aHash-test")]

# 해밍거리 임계값 (0 = 완전 동일, 1~5 매우 유사)
HAM_THR = 4
ah_leaks = []
# 간단 비교: test의 각 샘플을 train 일부와 비교(속도 위해 소표본 비교)
TRAIN_COMPARE_CAP = 2000
train_subset = train_ahashes if len(train_ahashes) <= TRAIN_COMPARE_CAP else random.sample(train_ahashes, TRAIN_COMPARE_CAP)

for tp, th in tqdm(test_ahashes, desc="aHash-compare"):
    if th is None:
        continue
    best_d, best_p = 999, None
    for rp, rh in train_subset:
        if rh is None:
            continue
        d = hamming(th, rh)
        if d < best_d:
            best_d, best_p = d, rp
            if d == 0: break
    if best_d <= HAM_THR:
        ah_leaks.append((tp, best_p, best_d))

df_ah = pd.DataFrame(ah_leaks, columns=["test_path","train_path","hamming"])
out_ah = os.path.join(WORK_DIR, "leak_perceptual_ahash.csv")
df_ah.to_csv(out_ah, index=False, encoding="utf-8-sig")
print(f"🔎 Perceptual Hash 유사(≤{HAM_THR}): {len(df_ah)}건 → {out_ah}")

# ---------- 4) SSIM 재시도 (임계값 낮추고 샘플↑) ----------
def ssim_gray(p, target_size=None):
    im = cv2.imread(p, cv2.IMREAD_GRAYSCALE)
    if im is None: return None
    if target_size:
        im = cv2.resize(im, target_size, interpolation=cv2.INTER_AREA)
    return im

# 샘플 수/임계값 조정
N_TEST_SSIM  = min(100, len(test_files))
N_TRAIN_SSIM = min(1000, len(train_files))
test_s = random.sample(test_files, N_TEST_SSIM) if len(test_files) > N_TEST_SSIM else test_files
train_s= random.sample(train_files, N_TRAIN_SSIM) if len(train_files) > N_TRAIN_SSIM else train_files
SSIM_THR = 0.97

ssim_hits = []
for tp in tqdm(test_s, desc="SSIM-test"):
    ti = ssim_gray(tp, target_size=(320,320))
    if ti is None:
        continue
    best_s, best_p = -1.0, None
    for rp in train_s:
        ri = ssim_gray(rp, target_size=(320,320))
        if ri is None:
            continue
        sc = ssim(ti, ri)
        if sc > best_s:
            best_s, best_p = sc, rp
    if best_s >= SSIM_THR:
        ssim_hits.append((tp, best_p, float(best_s)))

df_ss = pd.DataFrame(ssim_hits, columns=["test_path","train_path","ssim"])
out_ss = os.path.join(WORK_DIR, "leak_ssim_strict097.csv")
df_ss.to_csv(out_ss, index=False, encoding="utf-8-sig")
print(f"🔎 SSIM≥{SSIM_THR}: {len(df_ss)}건 → {out_ss}")

print("\n요약:")
print(f"- 파일명 stem 중복: {len(df_name)}")
print(f"- MD5 완전 동일(리사이즈 {RESIZE_TO}): {len(df_hash)}")
print(f"- Perceptual Hash 유사(≤{HAM_THR}): {len(df_ah)}")
print(f"- SSIM≥{SSIM_THR}: {len(df_ss)}")

TEST_DIR : ./data/ai05-level1-project/test_images
TRAIN_DIR: ./data/ai05-level1-project/train_images
#test=843  #train=1489
🔎 파일명 stem 중복: 0건 → ./work_yolov8/leak_name_stem_overlap.csv


Hash-test: 100%|██████████| 843/843 [00:31<00:00, 26.98it/s]


🔎 MD5(리사이즈 (256, 256)) 완전 동일: 838건 → ./work_yolov8/leak_exact_hash_md5.csv


aHash-compare: 100%|██████████| 843/843 [00:02<00:00, 287.64it/s]


🔎 Perceptual Hash 유사(≤4): 841건 → ./work_yolov8/leak_perceptual_ahash.csv


SSIM-test: 100%|██████████| 100/100 [1:11:36<00:00, 42.96s/it]

🔎 SSIM≥0.97: 65건 → ./work_yolov8/leak_ssim_strict097.csv

요약:
- 파일명 stem 중복: 0
- MD5 완전 동일(리사이즈 (256, 256)): 838
- Perceptual Hash 유사(≤4): 841
- SSIM≥0.97: 65


In [ ]:
from pathlib import Path
import zipfile, time
from google.colab import files

# 1️⃣ 폴더 경로 지정 (필요 시 절대경로로 수정)
ROOT = Path("./work_yolov8")

# 2️⃣ 존재 확인
assert ROOT.exists(), f"폴더 없음: {ROOT.resolve()}"

# 3️⃣ CSV 찾기
csv_files = list(ROOT.rglob("*.csv"))
assert csv_files, f"CSV 없음: {ROOT.resolve()}"

# 4️⃣ ZIP으로 묶기
zip_path = Path(f"{ROOT.name}_csv_{int(time.time())}.zip")
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for f in csv_files:
        zf.write(f, arcname=f.relative_to(ROOT))

# 5️⃣ Colab에서 바로 다운로드
files.download(zip_path.as_posix())

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 과거 블랙리스트 제거 로직

In [ ]:
# ===== B-1: 경로/모듈 세팅 + 블랙리스트(누수) 로드 =====
import os, json, shutil, glob, random
from pathlib import Path
import pandas as pd
from PIL import Image
from pycocotools.coco import COCO

# 프로젝트 경로
DATA_ROOT = "./data/ai05-level1-project"
IMG_DIR   = os.path.join(DATA_ROOT, "train_images")
TEST_DIR  = os.path.join(DATA_ROOT, "test_images")

WORK_DIR  = "./work_yolov8"
YOLO_DIR  = os.path.join(WORK_DIR, "yolo_dataset")
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(YOLO_DIR, exist_ok=True)

# A셀에서 저장된 누수 결과 목록(있으면 자동 병합)
leak_csv_candidates = [
    # "leak_ssim_optimized_wide.csv",
    # "leak_ssim_optimized.csv",
    "leak_perceptual_ahash.csv",
    "leak_exact_hash_md5.csv               ",
    "leak_ssim_strict097.csv"
]
blacklist_csvs = [os.path.join(WORK_DIR, n) for n in leak_csv_candidates if os.path.isfile(os.path.join(WORK_DIR, n))]

# 블랙리스트(학습에서 제외할 train 파일명)
black_train = set()
for p in blacklist_csvs:
    df = pd.read_csv(p)
    # 열 이름 유연 처리
    for col in ["train_path","train","path_train"]:
        if col in df.columns:
            black_train.update(os.path.basename(str(v)) for v in df[col].dropna().tolist())
            break

print(f"🧹 블랙리스트(학습 제외) 총 {len(black_train)}개 수집")

🧹 블랙리스트(학습 제외) 총 782개 수집


# 신규 블랙리스트 제거

In [ ]:
# ===== B-1: 블랙리스트(누수) CSV 통합 로더 =====
import os
import pandas as pd

# (옵션) blacklist_csvs가 미정의라면 기본 후보에서 자동 탐색
try:
    blacklist_csvs  # 존재 확인
except NameError:
    WORK_DIR = "./work_yolov8"
    os.makedirs(WORK_DIR, exist_ok=True)
    leak_csv_candidates = [
        # "leak_ssim_optimized_wide.csv",
        # "leak_ssim_optimized.csv",
        # "leak_perceptual_ahash.csv",
        # "leak_exact_hash_md5.csv",
        "leak_ssim_strict097.csv",
        # "leak_name_stem_overlap.csv",
    ]
    all_candidates = leak_csv_candidates
    blacklist_csvs = [
        (os.path.join(WORK_DIR, n) if not n.startswith("/mnt/") else n)
        for n in all_candidates
        if os.path.isfile(os.path.join(WORK_DIR, n) if not n.startswith("/mnt/") else n)
    ]

print(f"📂 탐색된 블랙리스트 CSV 파일 {len(blacklist_csvs)}개")
for i, path in enumerate(blacklist_csvs, 1):
    print(f"   {i:02d}. {os.path.basename(path)}")

# --- 블랙리스트(학습 제외할 train 파일명) ---
black_train = set()

for csv_path in blacklist_csvs:
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"⚠️ CSV 읽기 실패: {csv_path} ({e})")
        continue

    # 열 이름을 유연하게 감지
    target_col = None
    for col in ["train_path", "train", "path_train", "trainfile", "path_tr"]:
        if col in df.columns:
            target_col = col
            break

    if target_col is None:
        print(f"ℹ️ train 경로 열을 찾지 못함: {os.path.basename(csv_path)} → skip")
        continue

    # 경로 문자열 정리 후 basename만 추출
    vals = []
    for v in df[target_col].dropna().tolist():
        if not isinstance(v, str):
            v = str(v)
        v = v.strip()
        if not v:
            continue
        bn = os.path.basename(v.replace("\\", "/"))
        if bn:
            vals.append(bn)

    before = len(black_train)
    black_train.update(vals)
    added = len(black_train) - before
    print(f"✅ {os.path.basename(csv_path)} → {len(vals)}개 읽음 / {added}개 신규 추가")

# --- 결과 요약 ---
print(f"\n🧹 블랙리스트(학습 제외) 총 {len(black_train)}개 수집 완료")
preview = list(sorted(black_train))[:5]
print("예시 5개:", preview)

# (옵션) TXT로 저장하고 싶으면 아래 주석 해제
# out_txt = "./work_yolov8/black_train_basenames.txt"
# with open(out_txt, "w", encoding="utf-8") as f:
#     for bn in sorted(black_train):
#         f.write(bn + "\n")
# print(f"📝 저장 완료: {out_txt}")

📂 탐색된 블랙리스트 CSV 파일 2개
   01. leak_perceptual_ahash.csv
   02. leak_ssim_strict097.csv
✅ leak_perceptual_ahash.csv → 841개 읽음 / 775개 신규 추가
✅ leak_ssim_strict097.csv → 65개 읽음 / 7개 신규 추가

🧹 블랙리스트(학습 제외) 총 782개 수집 완료
예시 5개: ['K-001900-010224-016551-031705_0_2_0_2_75_000_200.png', 'K-001900-010224-016551-033009_0_2_0_2_70_000_200.png', 'K-001900-010224-016551-033009_0_2_0_2_75_000_200.png', 'K-001900-016548-018110-021026_0_2_0_2_90_000_200.png', 'K-001900-016548-018110-027926_0_2_0_2_70_000_200.png']


In [ ]:
# ===== B-2: 병합된 COCO(JSON) 로드 =====
MERGED_JSON = os.path.join(WORK_DIR, "train_all_robust.json")  # 앞서 만든 병합 산출물
if not os.path.isfile(MERGED_JSON):
    raise FileNotFoundError("merged_all_robust.json을 먼저 생성해 주세요. (셀: 병합 단계)")

with open(MERGED_JSON, "r") as f:
    merged = json.load(f)

print("📦 merged_all_robust.json 로드 완료")
print(" - images:", len(merged["images"]))
print(" - anns  :", len(merged["annotations"]))
print(" - cats  :", len(merged["categories"]))

📦 merged_all_robust.json 로드 완료
 - images: 1489
 - anns  : 4524
 - cats  : 73


In [ ]:
# ===== B-3: 블랙리스트 기반 필터링 =====
def file_name(im):  # COCO image dict → file_name
    return im.get("file_name") or im.get("imgfile") or ""

keep_img_ids = set()
for im in merged["images"]:
    fn = os.path.basename(file_name(im))
    if fn in black_train:
        continue  # 누수 의심 → 학습에서 제외
    keep_img_ids.add(im["id"])

filtered = {
    "images": [im for im in merged["images"] if im["id"] in keep_img_ids],
    "annotations": [a for a in merged["annotations"] if a["image_id"] in keep_img_ids],
    "categories": merged["categories"]
}
print("✅ 필터링 완료")
print(" - images:", len(filtered["images"]))
print(" - anns  :", len(filtered["annotations"]))
print(" - cats  :", len(filtered["categories"]))

✅ 필터링 완료
 - images: 707
 - anns  : 2543
 - cats  : 73


In [ ]:
# =========================================================
# COCO → Train/Val 분할(클래스 최소 커버 + stem 중복 방지) 전체 코드
# =========================================================
import os, json, random
from pathlib import Path
from collections import defaultdict

# -------------------------------
# 0) 기본 설정
# -------------------------------
SEED = 42
random.seed(SEED)

# WORK_DIR: 출력 JSON이 저장될 폴더
WORK_DIR = os.path.abspath("./work_yolov8")
os.makedirs(WORK_DIR, exist_ok=True)

# SOURCE_JSON: 단일 COCO 주석 파일(이미지가 모두 포함된 one-file)
# - 이미 메모리에 filtered(dict)가 있으면 그걸 사용하고,
# - 없으면 이 파일을 읽어 filtered로 사용
SOURCE_JSON = os.path.abspath("./data/ai05-level1-project/all.json")  # 필요시 경로 수정

# 분할 파라미터
K_PER_CLASS_VAL = 1     # 각 클래스가 최소 K개 이상 validation에 들어가도록 시도 (0~1 추천)
VAL_RATIO_TARGET = 0.20 # 최종적으로 전체의 20%를 validation으로 맞춤(필요시 수정)

# 저장 경로
TRAIN_JSON = os.path.join(WORK_DIR, "train.json")
VAL_JSON   = os.path.join(WORK_DIR, "val.json")


# -------------------------------
# 1) 유틸 함수
# -------------------------------
def load_coco_dict(path):
    with open(path, "r", encoding="utf-8") as f:
        d = json.load(f)
    # COCO 필수 키 체크
    for k in ["images", "annotations", "categories"]:
        if k not in d:
            raise ValueError(f"COCO key '{k}' missing in {path}")
    return d

def file_name(im):
    # COCO images[i]["file_name"]가 경로인 경우도 있으므로 basename 사용
    return os.path.basename(im.get("file_name", ""))

def stem(p):
    return Path(p).stem

def dump_coco_subset(src_coco_dict, keep_img_ids: set, out_path: str, reindex_ann: bool=True):
    """src_coco_dict에서 선택 이미지/어노로 서브셋 저장"""
    # 1) 이미지 필터
    img_keep = [im for im in src_coco_dict["images"] if im["id"] in keep_img_ids]
    keep_set = {im["id"] for im in img_keep}
    # 2) 어노테이션 필터
    anns_keep = [a for a in src_coco_dict["annotations"] if a.get("image_id") in keep_set]
    # 3) (선택) 어노 id 재부여
    if reindex_ann:
        for i, a in enumerate(anns_keep, 1):
            a["id"] = i
    out = {
        "images": img_keep,
        "annotations": anns_keep,
        "categories": src_coco_dict["categories"],
    }
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2)
    print(f"✅ saved {out_path} | images={len(img_keep)} anns={len(anns_keep)}")


# -------------------------------
# 2) 입력 COCO dict 확보
# -------------------------------
if "filtered" in globals() and isinstance(filtered, dict):
    coco_dict = filtered
    print("ℹ️ using existing 'filtered' dict from memory")
else:
    coco_dict = load_coco_dict(SOURCE_JSON)
    print(f"ℹ️ loaded source COCO:", SOURCE_JSON)

images = coco_dict["images"]
annotations = coco_dict["annotations"]
categories = coco_dict["categories"]

# -------------------------------
# 3) 분할 준비 (이미지별 포함 클래스, id↔파일명 맵)
# -------------------------------
img_to_cats = defaultdict(set)
for a in annotations:
    cid = a.get("category_id")
    iid = a.get("image_id")
    if iid is not None and cid is not None:
        img_to_cats[iid].add(cid)

id2fname = {im["id"]: file_name(im) for im in images}
all_img_ids = [im["id"] for im in images]
random.shuffle(all_img_ids)

N_total = len(all_img_ids)
target_val = max(1, int(round(VAL_RATIO_TARGET * N_total)))
print(f"📊 total images: {N_total} | target val: {target_val} ({VAL_RATIO_TARGET*100:.1f}%)")

# -------------------------------
# 4) 1단계: 클래스 최소 커버 확보 (과도하게 커지지 않게 10%에서 보호)
# -------------------------------
val_ids = set()
per_class_count = defaultdict(int)

for iid in all_img_ids:
    cats_here = img_to_cats.get(iid, set())
    picked = False
    for c in cats_here:
        if per_class_count[c] < K_PER_CLASS_VAL:
            val_ids.add(iid)
            # 해당 이미지가 갖는 모든 클래스 카운트 증가
            for c2 in cats_here:
                per_class_count[c2] += 1
            picked = True
            break
    # 과도한 유출 방지: 초기 커버링 단계는 최대 10%까지만
    if len(val_ids) > 0.10 * N_total:
        break

# -------------------------------
# 5) 2단계: stem 중복 방지하며 나머지를 train으로
# -------------------------------
remain = [iid for iid in all_img_ids if iid not in val_ids]
used_stems = set(stem(id2fname[i]) for i in val_ids)

train_ids = []
for iid in remain:
    s = stem(id2fname[iid])
    if s in used_stems:
        # 같은 stem은 val에 이미 들어갔으니 train에 넣지 않음
        continue
    train_ids.append(iid)
    used_stems.add(s)

# -------------------------------
# 6) 3단계: validation 목표 비율까지 이동
# -------------------------------
while len(val_ids) < target_val and len(train_ids) > 0:
    val_ids.add(train_ids.pop())

tr_ids, va_ids = set(train_ids), set(val_ids)
print(f"✂️ Split 결과 → train:{len(tr_ids)} | val:{len(va_ids)} (총 {N_total})")

# -------------------------------
# 7) 저장
# -------------------------------
dump_coco_subset(coco_dict, tr_ids, TRAIN_JSON)
dump_coco_subset(coco_dict, va_ids, VAL_JSON)

# 참고: 이후 YOLO data.yaml은 기존 방식대로 이미지 폴더 기준으로 구성하면 되고,
# json 기반 학습을 쓸 계획이라면 data.yaml의 'train:'/'val:'을 각각 {TRAIN_JSON}/{VAL_JSON} 경로로 지정해주면 된다.

ℹ️ using existing 'filtered' dict from memory
📊 total images: 707 | target val: 141 (20.0%)
✂️ Split 결과 → train:566 | val:141 (총 707)
✅ saved /content/work_yolov8/train.json | images=566 anns=2038
✅ saved /content/work_yolov8/val.json | images=141 anns=505


## 과거 train.json

In [ ]:
# ===== B-5: train.json / val.json 저장 =====
TRAIN_JSON = os.path.join(DATA_ROOT, "train.json")
VAL_JSON   = os.path.join(DATA_ROOT, "val.json")

def save_subset(src, keep_ids, out_path):
    out = {
        "images": [im for im in src["images"] if im["id"] in keep_ids],
        "annotations": [a for a in src["annotations"] if a["image_id"] in keep_ids],
        "categories": src["categories"]
    }
    with open(out_path, "w") as f:
        json.dump(out, f, indent=2, ensure_ascii=False)

save_subset(filtered, tr_ids, TRAIN_JSON)
save_subset(filtered, va_ids, VAL_JSON)

print("✅ COCO split 저장 완료")
print(" -", TRAIN_JSON)
print(" -", VAL_JSON)

✅ COCO split 저장 완료
 - ./data/ai05-level1-project/train.json
 - ./data/ai05-level1-project/val.json


## 신규 train.json

In [ ]:
# ==== COCO split 결과 저장 ====
import os, json
from pathlib import Path

# ===== B-4: Train/Val 분할 (stem 중복 + 클래스 최소 커버 보장) =====
from collections import defaultdict

def stem(p): return Path(p).stem

# 이미지별 포함된 카테고리
img_to_cats = defaultdict(set)
for a in filtered["annotations"]:
    img_to_cats[a["image_id"]].add(a["category_id"])

# 클래스 최소 커버(Validation에 클래스별 K개 이상 들어가도록)
K_PER_CLASS_VAL = 1  # 너무 크게 잡으면 val이 커짐. 0~1 추천

all_img_ids = [im["id"] for im in filtered["images"]]
random.shuffle(all_img_ids)

# 1) 클래스 커버 먼저 확보
val_ids = set()
per_class_count = defaultdict(int)
for iid in all_img_ids:
    cats_here = img_to_cats.get(iid, set())
    chosen = False
    for c in cats_here:
        if per_class_count[c] < K_PER_CLASS_VAL:
            val_ids.add(iid)
            for c2 in cats_here:
                per_class_count[c2] += 1
            chosen = True
            break
    # 안전장치: val이 과도하게 커지면 중단(10%)
    if len(val_ids) > 0.10 * len(all_img_ids):
        break

# 2) 나머지를 95/5 비율로, 단 stem 중복은 한 쪽만
remain = [iid for iid in all_img_ids if iid not in val_ids]
id2fname = {im["id"]: file_name(im) for im in filtered["images"]}

used_stems = set(stem(id2fname[iid]) for iid in val_ids)
train_ids = []
for iid in remain:
    s = stem(id2fname[iid])
    if s in used_stems:
        continue
    train_ids.append(iid)
    used_stems.add(s)

# 3) 만약 val이 너무 작으면 train에서 조금 이동
target_val = max(1, int(0.2 * len(all_img_ids)))
while len(val_ids) < target_val and len(train_ids) > 0:
    val_ids.add(train_ids.pop())

tr_ids, va_ids = set(train_ids), set(val_ids)

print(f"✂️ Split 결과 → train:{len(tr_ids)} | val:{len(va_ids)} (총 {len(all_img_ids)})")

# 저장 경로 (원하는 위치로 바꿔도 됨)
TRAIN_JSON = os.path.join(WORK_DIR, "train.json")
VAL_JSON   = os.path.join(WORK_DIR, "val.json")

def dump_coco_subset(src_coco_dict, keep_img_ids: set, out_path: str, reindex_ann: bool=True):
    # 1) 이미지 필터
    img_keep = [im for im in src_coco_dict["images"] if im["id"] in keep_img_ids]
    keep_set = {im["id"] for im in img_keep}

    # 2) 어노테이션 필터
    anns_keep = [a for a in src_coco_dict["annotations"] if a.get("image_id") in keep_set]

    # 3) (선택) 어노테이션 id 재부여 (1..N)
    if reindex_ann:
        for i, a in enumerate(anns_keep, 1):
            a["id"] = i

    out = {
        "images": img_keep,
        "annotations": anns_keep,
        "categories": src_coco_dict["categories"],
    }
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2)
    print(f"✅ saved {out_path} | images={len(img_keep)} anns={len(anns_keep)}")

# 위에서 만든 tr_ids, va_ids를 그대로 사용
dump_coco_subset(filtered, tr_ids, TRAIN_JSON)
dump_coco_subset(filtered, va_ids, VAL_JSON)


✂️ Split 결과 → train:566 | val:141 (총 707)
✅ saved ./work_yolov8/train.json | images=566 anns=2038
✅ saved ./work_yolov8/val.json | images=141 anns=505


In [ ]:
# =========================================================
# COCO → Train/Val 분할(4가지 비율) + 클래스 최소 커버 + stem 중복 방지
# =========================================================
import os, json, random, collections
from pathlib import Path
from collections import defaultdict

# -------------------------------
# 0) 기본 설정
# -------------------------------
WORK_DIR = os.path.abspath("./work_yolov8")
os.makedirs(WORK_DIR, exist_ok=True)

# 단일 COCO 주석(전체 이미지 포함 one-file). 메모리에 filtered가 있으면 그걸 우선 사용
SOURCE_JSON = os.path.abspath("./data/ai05-level1-project/all.json")  # 필요시 수정

# 원하는 검증 비율 목록(4가지)
VAL_RATIOS = [0.20, 0.15, 0.10, 0.05]   # 80/20, 85/15, 90/10, 95/5
# 각 분할에 사용할 시드(재현성). 비율 개수와 길이를 맞추면 됨
SEEDS = [42, 43, 44, 45]

# 각 클래스가 최소 몇 장 이상 VAL에 들어가도록 시도(0~1 권장)
K_PER_CLASS_VAL = 1

# 저장 루트: /work_yolov8/splits/R20/, R15/, R10/, R05/ 구조로 저장
SPLIT_ROOT = os.path.join(WORK_DIR, "splits")
os.makedirs(SPLIT_ROOT, exist_ok=True)

# -------------------------------
# 1) 유틸 함수
# -------------------------------
def load_coco_dict(path):
    with open(path, "r", encoding="utf-8") as f:
        d = json.load(f)
    for k in ["images", "annotations", "categories"]:
        if k not in d:
            raise ValueError(f"COCO key '{k}' missing in {path}")
    return d

def file_name(im):
    return os.path.basename(im.get("file_name", ""))

def stem(p):
    return Path(p).stem

def dump_coco_subset(src, keep_img_ids: set, out_path: str, reindex_ann: bool=True):
    img_keep = [im for im in src["images"] if im["id"] in keep_img_ids]
    keep_ids = {im["id"] for im in img_keep}
    anns_keep = [a for a in src["annotations"] if a.get("image_id") in keep_ids]
    if reindex_ann:
        for i, a in enumerate(anns_keep, 1):
            a["id"] = i
    out = {"images": img_keep, "annotations": anns_keep, "categories": src["categories"]}
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(out, f, ensure_ascii=False, indent=2)
    print(f"✅ saved {out_path} | images={len(img_keep)} anns={len(anns_keep)}")

def count_per_class(src, img_ids):
    """클래스별 어노테이션 수 집계(리포팅용)"""
    keep = set(img_ids)
    cnt = collections.Counter()
    for a in src["annotations"]:
        if a.get("image_id") in keep:
            cnt[a["category_id"]] += 1
    return dict(cnt)

# -------------------------------
# 2) 입력 COCO dict 확보
# -------------------------------
if "filtered" in globals() and isinstance(filtered, dict):
    coco = filtered
    print("ℹ️ using existing 'filtered' dict from memory")
else:
    coco = load_coco_dict(SOURCE_JSON)
    print(f"ℹ️ loaded source COCO: {SOURCE_JSON}")

images = coco["images"]
annotations = coco["annotations"]
categories = coco["categories"]

# ---------- 공통 전처리 ----------
# 이미지별 포함된 카테고리
img_to_cats = defaultdict(set)
for a in annotations:
    cid = a.get("category_id"); iid = a.get("image_id")
    if iid is not None and cid is not None:
        img_to_cats[iid].add(cid)

id2fname = {im["id"]: file_name(im) for im in images}
all_img_ids_master = [im["id"] for im in images]
N_total = len(all_img_ids_master)

print(f"📊 total images: {N_total}")

# -------------------------------
# 3) 비율별로 4개 분할 실행
# -------------------------------
REPORT = []  # 요약 리포트

for ratio, seed in zip(VAL_RATIOS, SEEDS):
    random.seed(seed)

    # 폴더 태그: 0.20 -> R20, 0.15 -> R15 ...
    tag = f"R{int(round(ratio*100)):02d}"
    out_dir = os.path.join(SPLIT_ROOT, tag)
    os.makedirs(out_dir, exist_ok=True)

    # 3-1) 전체 id 셔플
    all_img_ids = list(all_img_ids_master)
    random.shuffle(all_img_ids)

    # 목표 val 크기
    target_val = max(1, int(round(ratio * N_total)))
    print(f"\n=== Split {tag} (val={ratio*100:.1f}% | seed={seed}) ===")
    print(f"target val size: {target_val}")

    # 3-2) 1단계: 클래스 최소 커버 확보(최대 10%까지)
    val_ids = set()
    per_class_count = defaultdict(int)
    for iid in all_img_ids:
        cats_here = img_to_cats.get(iid, set())
        picked = False
        for c in cats_here:
            if per_class_count[c] < K_PER_CLASS_VAL:
                val_ids.add(iid)
                for c2 in cats_here:
                    per_class_count[c2] += 1
                picked = True
                break
        if len(val_ids) > 0.10 * N_total:
            break

    # 3-3) 2단계: stem 중복 방지하며 나머지를 train으로
    remain = [iid for iid in all_img_ids if iid not in val_ids]
    used_stems = set(stem(id2fname[i]) for i in val_ids)

    train_ids = []
    for iid in remain:
        s = stem(id2fname[iid])
        if s in used_stems:
            continue
        train_ids.append(iid)
        used_stems.add(s)

    # 3-4) 3단계: validation 목표까지 이동
    while len(val_ids) < target_val and len(train_ids) > 0:
        val_ids.add(train_ids.pop())

    tr_ids, va_ids = set(train_ids), set(val_ids)
    print(f"✂️ Result {tag} → train:{len(tr_ids)} | val:{len(va_ids)}")

    # 3-5) 저장
    train_json = os.path.join(out_dir, "train.json")
    val_json   = os.path.join(out_dir, "val.json")
    dump_coco_subset(coco, tr_ids, train_json)
    dump_coco_subset(coco, va_ids, val_json)

    # 3-6) 리포트(클래스별 카운트)
    rep = {
        "tag": tag,
        "seed": seed,
        "val_ratio": ratio,
        "n_train": len(tr_ids),
        "n_val": len(va_ids),
        "train_per_class": count_per_class(coco, tr_ids),
        "val_per_class": count_per_class(coco, va_ids),
        "paths": {"train_json": train_json, "val_json": val_json},
    }
    REPORT.append(rep)

# -------------------------------
# 4) 요약 저장
# -------------------------------
summary_path = os.path.join(SPLIT_ROOT, "split_summary.json")
with open(summary_path, "w", encoding="utf-8") as f:
    json.dump(REPORT, f, ensure_ascii=False, indent=2)
print(f"\n📝 split summary saved: {summary_path}")

# ===== 출력 예시 =====
# /work_yolov8/splits/R20/train.json, val.json
# /work_yolov8/splits/R15/train.json, val.json
# /work_yolov8/splits/R10/train.json, val.json
# /work_yolov8/splits/R05/train.json, val.json

ℹ️ using existing 'filtered' dict from memory
📊 total images: 707

=== Split R20 (val=20.0% | seed=42) ===
target val size: 141
✂️ Result R20 → train:566 | val:141
✅ saved /content/work_yolov8/splits/R20/train.json | images=566 anns=2038
✅ saved /content/work_yolov8/splits/R20/val.json | images=141 anns=505

=== Split R15 (val=15.0% | seed=43) ===
target val size: 106
✂️ Result R15 → train:601 | val:106
✅ saved /content/work_yolov8/splits/R15/train.json | images=601 anns=2158
✅ saved /content/work_yolov8/splits/R15/val.json | images=106 anns=385

=== Split R10 (val=10.0% | seed=44) ===
target val size: 71
✂️ Result R10 → train:636 | val:71
✅ saved /content/work_yolov8/splits/R10/train.json | images=636 anns=2288
✅ saved /content/work_yolov8/splits/R10/val.json | images=71 anns=255

=== Split R05 (val=5.0% | seed=45) ===
target val size: 35
✂️ Result R05 → train:660 | val:47
✅ saved /content/work_yolov8/splits/R05/train.json | images=660 anns=2371
✅ saved /content/work_yolov8/splits/R05

### 배경 제거 전 coco -> yolo 변경

In [ ]:
# ===== COCO → YOLO (심플 + KeyError 고친 버전) =====
import os, json, shutil, glob
from pathlib import Path
from pycocotools.coco import COCO

# ---- 경로만 맞춰줘 ----
DATA_ROOT = "./data/ai05-level1-project"
IMG_DIR   = f"{DATA_ROOT}/train_images"

WORK_DIR = "./work_yolov8"
YOLO_DIR = f"{WORK_DIR}/yolo_dataset"

TRAIN_JSON = f"{WORK_DIR}/train.json"
VAL_JSON   = f"{WORK_DIR}/val.json"

os.makedirs(f"{YOLO_DIR}/images/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/images/val",   exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/train", exist_ok=True)
os.makedirs(f"{YOLO_DIR}/labels/val",   exist_ok=True)

# --- 0) 카테고리 매핑: train+val 합집합으로 생성 ---
coco_tr = COCO(TRAIN_JSON)
coco_va = COCO(VAL_JSON)

cat_ids_union = sorted(set(coco_tr.getCatIds()) | set(coco_va.getCatIds()))
cats_union    = coco_tr.loadCats(cat_ids_union)  # 이름은 train 기준으로 로드해도 무방
names         = [c["name"] for c in cats_union]

# COCO category_id -> YOLO class index
catid_to_idx = {cid: i for i, cid in enumerate(cat_ids_union)}
# 저장(키는 문자열로 저장되므로, 읽을 땐 int로 캐스팅해야 함)
with open(f"{YOLO_DIR}/catid_to_idx.json","w",encoding="utf-8") as f:
    json.dump(catid_to_idx, f, ensure_ascii=False, indent=2)

# data.yaml 생성
with open(f"{YOLO_DIR}/data_abs.yaml", "w", encoding="utf-8") as f:
    f.write(f"train: {os.path.abspath(YOLO_DIR)}/images/train\n")
    f.write(f"val: {os.path.abspath(YOLO_DIR)}/images/val\n\n")
    f.write(f"nc: {len(names)}\n")
    f.write("names: " + json.dumps(names, ensure_ascii=False) + "\n")

def _safe_locate(src_root, filename):
    p = os.path.join(src_root, filename)
    if os.path.isfile(p):
        return p
    # 하위폴더 탐색(최초 1개만 사용)
    hits = glob.glob(os.path.join(src_root, "**", Path(filename).name), recursive=True)
    return hits[0] if hits else None

def _convert_one_split(coco_json, split):
    coco = COCO(coco_json)
    img_dir_out = f"{YOLO_DIR}/images/{split}"
    lbl_dir_out = f"{YOLO_DIR}/labels/{split}"

    for iid in coco.getImgIds():
        im = coco.loadImgs([iid])[0]
        fn = im["file_name"]
        src = _safe_locate(IMG_DIR, fn)
        if src is None:
            print("skip(missing):", fn)
            continue

        dst_img = os.path.join(img_dir_out, Path(fn).name)
        if not os.path.isfile(dst_img):
            shutil.copy2(src, dst_img)

        W, H = im["width"], im["height"]
        anns = coco.loadAnns(coco.getAnnIds(imgIds=[iid]))

        # 라벨 저장
        with open(os.path.join(lbl_dir_out, Path(fn).stem + ".txt"), "w") as f:
            for a in anns[:4]:  # 필요시 제한 해제
                cid = int(a["category_id"])
                if cid not in catid_to_idx:
                    # (이론상 합집합 매핑이니 여기 안 걸려야 함)
                    # 혹시라도 걸리면 안전하게 건너뜀
                    # print(f"warn: category {cid} not in mapping, skip")
                    continue
                x, y, w, h = a["bbox"]
                xc = (x + w/2) / W
                yc = (y + h/2) / H
                f.write(f"{catid_to_idx[cid]} {xc} {yc} {w/W} {h/H}\n")

# 실행
_convert_one_split(TRAIN_JSON, "train")
_convert_one_split(VAL_JSON,   "val")

print("✅ 완료!")
print(" - images/train :", len(os.listdir(f'{YOLO_DIR}/images/train')))
print(" - images/val   :", len(os.listdir(f'{YOLO_DIR}/images/val')))
print(" - data yaml    :", f"{YOLO_DIR}/data_abs.yaml")

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
✅ 완료!
 - images/train : 1353
 - images/val   : 71
 - data yaml    : ./work_yolov8/yolo_dataset/data_abs.yaml


In [ ]:
# =========================================================
# Split별 COCO → YOLO 변환기 (분할마다 독립 YOLO 데이터셋 생성)
# - 입력: WORK_DIR/splits/<TAG>/{train.json, val.json}
# - 출력: WORK_DIR/yolo_dataset/<TAG>/{images,labels}/{train,val}, data_abs.yaml
# =========================================================
import os, json, glob, shutil
from pathlib import Path
from pycocotools.coco import COCO

# ---------- 경로 설정 ----------
WORK_DIR   = os.path.abspath("./work_yolov8")
SPLIT_DIR  = os.path.join(WORK_DIR, "splits")                 # R20/R15/R10/R05 폴더 있는 곳
YOLO_ROOT  = os.path.join(WORK_DIR, "yolo_dataset")           # 분할별 YOLO가 들어갈 루트
IMG_DIR    = os.path.abspath("./data/ai05-level1-project/train_images")  # 원본 이미지 루트

# 변환할 분할 태그들(필요에 맞게 수정)
SPLIT_TAGS = ["R20", "R15", "R10", "R05"]

# ---------- 유틸 ----------
def _safe_locate(src_root, filename):
    """src_root 아래에서 filename을 찾는다(하위폴더 포함). 없으면 None."""
    p = os.path.join(src_root, filename)
    if os.path.isfile(p):
        return p
    hits = glob.glob(os.path.join(src_root, "**", Path(filename).name), recursive=True)
    return hits[0] if hits else None

def _ensure_dirs(*paths):
    for p in paths:
        os.makedirs(p, exist_ok=True)

def _write_yaml(yolo_dir, class_names):
    data_yaml = os.path.join(yolo_dir, "data_abs.yaml")
    with open(data_yaml, "w", encoding="utf-8") as f:
        f.write(f"train: {os.path.abspath(os.path.join(yolo_dir, 'images', 'train'))}\n")
        f.write(f"val: {os.path.abspath(os.path.join(yolo_dir, 'images', 'val'))}\n\n")
        f.write(f"nc: {len(class_names)}\n")
        f.write("names: " + json.dumps(class_names, ensure_ascii=False) + "\n")
    return data_yaml

def _build_union_mapping(train_json, val_json):
    """train+val 합집합 category_id → yolo class index, names"""
    c_tr = COCO(train_json)
    c_va = COCO(val_json)
    union_ids = sorted(set(c_tr.getCatIds()) | set(c_va.getCatIds()))
    # 이름은 train 기준으로 로드해도 무방(동명일치 가정)
    names = [c["name"] for c in c_tr.loadCats(union_ids)]
    catid_to_idx = {int(cid): i for i, cid in enumerate(union_ids)}
    return catid_to_idx, names

def _convert_one_split(train_json, val_json, img_dir, yolo_dir, catid_to_idx):
    """COCO → YOLO (images/labels/{train,val}) 생성"""
    for split, coco_json in [("train", train_json), ("val", val_json)]:
        coco = COCO(coco_json)
        img_out = os.path.join(yolo_dir, "images", split)
        lbl_out = os.path.join(yolo_dir, "labels", split)
        _ensure_dirs(img_out, lbl_out)

        for iid in coco.getImgIds():
            im = coco.loadImgs([iid])[0]
            fn = im["file_name"]
            src = _safe_locate(img_dir, fn)
            if src is None:
                print(f"skip(missing): {fn}")
                continue

            # 이미지 복사(중복 방지)
            dst_img = os.path.join(img_out, Path(fn).name)
            if not os.path.isfile(dst_img):
                shutil.copy2(src, dst_img)

            # 라벨 작성
            W, H = im["width"], im["height"]
            anns = coco.loadAnns(coco.getAnnIds(imgIds=[iid]))
            with open(os.path.join(lbl_out, Path(fn).stem + ".txt"), "w") as f:
                for a in anns:   # 필요 시 [:4]로 제한 가능
                    cid = int(a["category_id"])
                    if cid not in catid_to_idx:
                        # 합집합 매핑에 없으면 스킵(이론상 거의 없음)
                        continue
                    x, y, w, h = a["bbox"]
                    xc = (x + w/2) / W
                    yc = (y + h/2) / H
                    f.write(f"{catid_to_idx[cid]} {xc} {yc} {w/W} {h/H}\n")

# ---------- 메인 루프 ----------
os.makedirs(YOLO_ROOT, exist_ok=True)

for tag in SPLIT_TAGS:
    split_path = os.path.join(SPLIT_DIR, tag)
    train_json = os.path.join(split_path, "train.json")
    val_json   = os.path.join(split_path, "val.json")
    assert os.path.isfile(train_json) and os.path.isfile(val_json), f"{tag}: train/val json이 없습니다."

    yolo_dir = os.path.join(YOLO_ROOT, tag)
    _ensure_dirs(yolo_dir, os.path.join(yolo_dir, "images"), os.path.join(yolo_dir, "labels"))

    # 1) 카테고리 매핑(합집합) 생성 + 저장
    catid_to_idx, names = _build_union_mapping(train_json, val_json)
    with open(os.path.join(yolo_dir, "catid_to_idx.json"), "w", encoding="utf-8") as f:
        json.dump(catid_to_idx, f, ensure_ascii=False, indent=2)

    # 2) data.yaml 작성
    data_yaml = _write_yaml(yolo_dir, names)

    # 3) 변환 실행
    print(f"\n=== [{tag}] COCO → YOLO 변환 시작 ===")
    _convert_one_split(train_json, val_json, IMG_DIR, yolo_dir, catid_to_idx)

    # 4) 결과 간단 리포트
    n_tr_img = len(os.listdir(os.path.join(yolo_dir, "images", "train")))
    n_va_img = len(os.listdir(os.path.join(yolo_dir, "images", "val")))
    print(f"✅ [{tag}] 완료 | train images: {n_tr_img} | val images: {n_va_img}")
    print(f"📝 data.yaml: {data_yaml}")

    # (선택) Colab/Ultralytics 기본 경로 호환 심볼릭 링크
    try:
        os.makedirs("/content/datasets", exist_ok=True)
        link_dir = os.path.join("/content/datasets", f"yolo_dataset_{tag}")
        if os.path.islink(link_dir) or os.path.exists(link_dir):
            # 기존 링크/폴더 정리
            try: os.remove(link_dir)
            except IsADirectoryError:
                import shutil as _sh; _sh.rmtree(link_dir)
        os.symlink(yolo_dir, link_dir)
        print(f"🔗 symlink: {link_dir} -> {yolo_dir}")
    except Exception:
        pass

print("\n🎯 모든 분할 변환 완료.")
print(f"루트: {YOLO_ROOT}")
# 예) 학습 시:
# model.train(data=f"{YOLO_ROOT}/R20/data_abs.yaml", imgsz=640, epochs=…)
# model.train(data=f"{YOLO_ROOT}/R15/data_abs.yaml", imgsz=640, epochs=…)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

=== [R20] COCO → YOLO 변환 시작 ===
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
✅ [R20] 완료 | train images: 566 | val images: 141
📝 data.yaml: /content/work_yolov8/yolo_dataset/R20/data_abs.yaml
🔗 symlink: /content/datasets/yolo_dataset_R20 -> /content/work_yolov8/yolo_dataset/R20
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

=== [R15] COCO → YOLO 변환 시작 ===
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
✅ [R15] 완료 | train images: 601 | val images: 106
📝 data.yaml: /conten

### 배경제거 coco -> yolo

In [ ]:
# =========================================================
# 6) COCO → YOLO 변환 (+ 배경제거 포함)
# =========================================================
import os, shutil
from pathlib import Path
from pycocotools.coco import COCO
from PIL import Image
from rembg import remove
from tqdm import tqdm

def convert_coco_to_yolo(TRAIN_JSON, VAL_JSON, IMG_DIR, YOLO_DIR, remove_bg=False):
    """
    COCO 포맷 → YOLO 포맷 변환 (옵션: rembg로 배경 제거)
    - TRAIN_JSON, VAL_JSON: COCO annotation 파일 경로
    - IMG_DIR: 원본 이미지 디렉토리
    - YOLO_DIR: 변환 결과 저장 폴더
    - remove_bg=True 시 rembg(U²-Net)으로 배경 제거 후 저장
    """
    os.makedirs(YOLO_DIR, exist_ok=True)
    for s in ["train", "val"]:
        os.makedirs(f"{YOLO_DIR}/images/{s}", exist_ok=True)
        os.makedirs(f"{YOLO_DIR}/labels/{s}", exist_ok=True)

    coco = COCO(TRAIN_JSON)
    cat_ids = sorted(coco.getCatIds())
    catid_to_idx = {cid: i for i, cid in enumerate(cat_ids)}

    # ----------------------------
    # Split별 변환 (train / val)
    # ----------------------------
    for split, src_json in [("train", TRAIN_JSON), ("val", VAL_JSON)]:
        coco = COCO(src_json)
        img_ids = coco.getImgIds()

        print(f"\n📂 Converting {split} set ({len(img_ids)} images)...")

        for iid in tqdm(img_ids):
            im = coco.loadImgs([iid])[0]
            src = os.path.join(IMG_DIR, im["file_name"])
            dst = f"{YOLO_DIR}/images/{split}/{im['file_name']}"

            # ⚠️ 이미지 존재 확인
            if not os.path.isfile(src):
                print(f"⚠️ skip missing image: {src}")
                continue

            # ----------------------------
            # 🧹 [옵션] rembg로 배경 제거
            # ----------------------------
            if remove_bg:
                try:
                    img = Image.open(src).convert("RGBA")
                    result = remove(img)
                    # 검은 배경으로 채워서 RGB로 저장
                    bg = Image.new("RGB", result.size, (0, 0, 0))
                    bg.paste(result, mask=result.split()[3])  # A 채널 기준
                    bg.save(dst)
                except Exception as e:
                    print(f"⚠️ skip bg-remove error on {src}: {e}")
                    shutil.copy(src, dst)
            else:
                shutil.copy(src, dst)

            # ----------------------------
            # 📄 YOLO 라벨 생성
            # ----------------------------
            anns = coco.loadAnns(coco.getAnnIds(imgIds=[iid]))
            with open(f"{YOLO_DIR}/labels/{split}/{Path(im['file_name']).stem}.txt", "w") as f:
                for a in anns[:4]:
                    x, y, w, h = a["bbox"]
                    xc, yc = (x + w / 2) / im["width"], (y + h / 2) / im["height"]
                    f.write(f"{catid_to_idx[a['category_id']]} {xc} {yc} {w/im['width']} {h/im['height']}\n")

    # ----------------------------
    # YAML 파일 생성
    # ----------------------------
    yaml = f"{YOLO_DIR}/data_abs.yaml"
    with open(yaml, "w") as f:
        f.write(
            f"train: {YOLO_DIR}/images/train\n"
            f"val: {YOLO_DIR}/images/val\n"
            f"nc: {len(cat_ids)}\n"
            f"names: {cat_ids}"
        )

    print("\n✅ YOLO 변환 및 배경제거 완료:", yaml)
    return yaml, catid_to_idx


data_yaml_abs, catid_to_idx = convert_coco_to_yolo(TRAIN_JSON, VAL_JSON, IMG_DIR, YOLO_DIR, remove_bg=True)
print("✅ YOLO 변환 완료:", data_yaml_abs)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!

📂 Converting train set (578 images)...


100%|██████████| 578/578 [22:02<00:00,  2.29s/it]


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

📂 Converting val set (45 images)...


100%|██████████| 45/45 [01:42<00:00,  2.29s/it]


✅ YOLO 변환 및 배경제거 완료: ./work_yolov8/yolo_dataset/data_abs.yaml
✅ YOLO 변환 완료: ./work_yolov8/yolo_dataset/data_abs.yaml


### 돌리기

In [ ]:
import os, yaml

DATA_YAML = "./work_yolov8/yolo_dataset/data_abs.yaml"

# 1️⃣ YAML 파일 로드
with open(DATA_YAML, "r") as f:
    y = yaml.safe_load(f)
print("🔹 YAML 내용:", y, "\n")

# 2️⃣ 경로 실제 존재 확인
def check_path(k, v):
    # path + 상대경로 조합 처리
    if k in ["train", "val"] and "path" in y and not os.path.isabs(v):
        joined = os.path.join(y["path"], v)
        print(f"{k} → {v}\n   joined: {joined}\n   exists:", os.path.exists(joined))
    else:
        print(f"{k} → {v}\n   exists:", os.path.exists(v))

for key in ["path", "train", "val"]:
    if key in y:
        check_path(key, y[key])

🔹 YAML 내용: {'train': './work_yolov8/yolo_dataset/images/train', 'val': './work_yolov8/yolo_dataset/images/val', 'nc': 73, 'names': [1899, 2482, 3350, 3482, 3543, 3742, 3831, 4377, 4542, 5093, 5885, 6191, 6562, 10220, 12080, 12246, 12419, 12777, 13394, 13899, 16231, 16261, 16547, 16550, 16687, 18109, 18146, 18356, 19231, 19551, 19606, 19860, 20013, 20237, 20876, 21025, 21324, 21770, 22073, 22346, 22361, 22626, 23202, 23222, 24849, 25366, 25437, 25468, 27652, 27732, 27776, 27925, 27992, 28762, 29344, 29450, 29666, 29870, 30307, 31704, 31862, 31884, 32309, 33008, 33207, 33877, 33879, 34596, 35205, 36636, 38161, 41767, 44198]} 

train → ./work_yolov8/yolo_dataset/images/train
   exists: True
val → ./work_yolov8/yolo_dataset/images/val
   exists: True


In [ ]:
import os, yaml

DATA_YAML = "./work_yolov8/yolo_dataset/data_abs.yaml"

with open(DATA_YAML, "r") as f:
    y = yaml.safe_load(f)

# 기존 names, nc는 그대로 보존하고 train/val만 절대경로로 교체
y["train"] = os.path.abspath(y["train"])
y["val"]   = os.path.abspath(y["val"])

with open(DATA_YAML, "w") as f:
    yaml.safe_dump(y, f, allow_unicode=True, sort_keys=False)

print("✅ data_abs.yaml updated:")
print("train:", y["train"])
print("val  :", y["val"])

✅ data_abs.yaml updated:
train: /content/work_yolov8/yolo_dataset/images/train
val  : /content/work_yolov8/yolo_dataset/images/val


In [ ]:
# ==== 안전 재시작 설정 ====
import os, glob, gc, shutil, torch, pandas as pd
from pathlib import Path
from ultralytics import YOLO

# --- 절대경로 체크 ---
DATA_YAML = os.path.abspath("./work_yolov8/yolo_dataset/data_abs.yaml")
WORK_DIR  = os.path.abspath("./work_yolov8")

print("✅ DATA_YAML:", DATA_YAML)
assert os.path.isfile(DATA_YAML), "data_abs.yaml 파일이 존재하지 않습니다."
assert os.path.isdir(WORK_DIR), "WORK_DIR 폴더가 존재하지 않습니다."

# --- 캐시 정리 ---
def clear_yolo_caches(yolo_dir):
    for p in [
        os.path.join(yolo_dir, "labels", "train.cache"),
        os.path.join(yolo_dir, "labels", "val.cache"),
    ]:
        if os.path.isfile(p):
            print("🧹 remove cache:", p)
            os.remove(p)

clear_yolo_caches(os.path.dirname(DATA_YAML))

# --- 리소스 정리 ---
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(False)

# --- Sweep 설정 ---
IMGSZ_SWEEP = 640
CANDIDATES = ["yolov8s.pt", "yolov8m.pt", "yolov8l.pt"]
SAFE_WORKERS = 2
SAFE_BATCH = 8
PATIENCE = 50
scores = []

# --- Sweep 루프 ---
for name in CANDIDATES:
    exp_name = f"sweep_{Path(name).stem}"
    run_dir = os.path.join(WORK_DIR, exp_name)
    weight_last = os.path.join(run_dir, "weights", "last.pt")
    weight_best = os.path.join(run_dir, "weights", "best.pt")

    print(f"\n=== RESILIENT SWEEP: {name} ===")
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    try:
        if os.path.isfile(weight_last):
            print("↩️ resume from:", weight_last)
            m = YOLO(weight_last)
            m.train(
                data=DATA_YAML, epochs=20, imgsz=IMGSZ_SWEEP,
                batch=SAFE_BATCH, workers=SAFE_WORKERS,
                device=0, project=WORK_DIR, name=exp_name,
                resume=True, verbose=True, patience=PATIENCE,
                mosaic=0.0, mixup=0.0, copy_paste=0.0,
                fliplr=0.5, flipud=0.0,
                degrees=0.0, translate=0.10, scale=0.50, shear=0.0,
                hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
            )
        else:
            m = YOLO(name)
            m.train(
                data=DATA_YAML, epochs=20, imgsz=IMGSZ_SWEEP,
                batch=SAFE_BATCH, workers=SAFE_WORKERS,
                device=0, project=WORK_DIR, name=exp_name,
                verbose=True, patience=PATIENCE,
                mosaic=0.0, mixup=0.0, copy_paste=0.0,
                fliplr=0.5, flipud=0.0,
                degrees=0.0, translate=0.10, scale=0.50, shear=0.0,
                hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
            )

        # 학습 성공 시 검증
        r = m.val(imgsz=IMGSZ_SWEEP, data=DATA_YAML, workers=SAFE_WORKERS)
        scores.append((name, float(r.box.map), float(r.box.map50), float(r.box.map75)))
        continue

    except Exception as e:
        print("⚠️ train/val error:", e)
        cand = weight_best if os.path.isfile(weight_best) else (
            weight_last if os.path.isfile(weight_last) else None
        )

        if cand:
            print("🧪 fallback evaluate:", cand)
            try:
                m = YOLO(cand)
                r = m.val(imgsz=IMGSZ_SWEEP, data=DATA_YAML, workers=SAFE_WORKERS)
                scores.append((name, float(r.box.map), float(r.box.map50), float(r.box.map75)))
            except Exception as e2:
                print("❌ fallback val error:", e2)
                scores.append((name, 0.0, 0.0, 0.0))
        else:
            scores.append((name, 0.0, 0.0, 0.0))

# --- 결과 저장 ---
df = pd.DataFrame(scores, columns=["model", "mAP@[.5:.95]", "mAP@.50", "mAP@.75"])
df_path = os.path.join(WORK_DIR, "sweep_results_imgsz640_resilient.csv")
df.to_csv(df_path, index=False, encoding="utf-8-sig")

print("\n스윕 결과:")
print(df.sort_values("mAP@[.5:.95]", ascending=False))

BEST_MODEL = df.sort_values("mAP@[.5:.95]", ascending=False).iloc[0]["model"]
print(f"\n✅ 최고 모델: {BEST_MODEL}")
print(f"📊 결과 CSV 저장 위치: {df_path}")

✅ DATA_YAML: /content/work_yolov8/yolo_dataset/data_abs.yaml

=== RESILIENT SWEEP: yolov8s.pt ===
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/work_yolov8/yolo_dataset/data_abs.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.0, multi_scale=False, name=sweep_yolov8s, nbs=64, n

## 베스트로 해서 어떻게 더 좋은 데이터 증강인지 파악

In [ ]:
# ===== Final 150-epoch Fine-tune (3-Stage, robust; 새 인스턴스 방식) =====
import os, glob, gc, torch, json, pandas as pd
from pathlib import Path
from ultralytics import YOLO

# --- 0) 경로/입력 보강 ---------------------------------------------------------
# 이전 셀에서 지정 안 됐다면 안전한 기본값으로 세팅
if 'WORK_DIR' not in globals():
    WORK_DIR = os.path.abspath("./work_yolov8")
if 'DATA_YAML' not in globals():
    DATA_YAML = os.path.abspath("./work_yolov8/yolo_dataset/data_abs.yaml")

WORK_DIR  = os.path.abspath(WORK_DIR)
DATA_YAML = os.path.abspath(DATA_YAML)

print("WORK_DIR :", WORK_DIR, "| exists:", os.path.isdir(WORK_DIR))
print("DATA_YAML:", DATA_YAML, "| exists:", os.path.isfile(DATA_YAML))
assert os.path.isdir(WORK_DIR),  "WORK_DIR 폴더가 없습니다."
assert os.path.isfile(DATA_YAML), "DATA_YAML 파일이 없습니다."

CSV_PATH = os.path.join(WORK_DIR, "sweep_results_imgsz640_resilient.csv")
assert os.path.isfile(CSV_PATH), f"스윕 결과 CSV가 없습니다: {CSV_PATH}"

# --- 1) 스윕 결과에서 베스트 모델 태그 읽고 시작 가중치 찾기 -------------------
df = pd.read_csv(CSV_PATH)
assert len(df) > 0, "스윕 결과가 비어있습니다."
best_row = df.sort_values("mAP@[.5:.95]", ascending=False).iloc[0]
best_model_name = str(best_row["model"]).strip()  # ex) 'yolov8s.pt', 'yolov8m.pt', ...
print("🏆 sweep best model tag:", best_model_name)

def find_run_dirs_for_model(model_name: str):
    stem = Path(model_name).stem  # 'yolov8s'
    patterns = [
        os.path.join(WORK_DIR, f"sweep_{stem}"),
        os.path.join(WORK_DIR, f"sweep_{stem}*"),     # sweep_yolov8s, sweep_yolov8s4 등
        os.path.join(WORK_DIR, f"**/sweep_{stem}*"),  # 하위폴더 재귀
    ]
    found = []
    for pat in patterns:
        found += glob.glob(pat, recursive=True)
    found = [d for d in found if os.path.isdir(os.path.join(d, "weights"))]
    found.sort(key=lambda d: os.path.getmtime(os.path.join(d, "weights")), reverse=True)
    return found

run_dirs = find_run_dirs_for_model(best_model_name)
weight_path = None
for rd in run_dirs:
    for w in ["best.pt", "last.pt"]:
        p = os.path.join(rd, "weights", w)
        if os.path.isfile(p):
            weight_path = p
            break
    if weight_path: break

# 폴백: 그래도 못 찾으면 WORK_DIR 전체에서 최신 best.pt
if weight_path is None:
    cands = glob.glob(os.path.join(WORK_DIR, "**", "weights", "best.pt"), recursive=True)
    cands.sort(key=os.path.getmtime, reverse=True)
    weight_path = cands[0] if cands else None

print("🏁 Start weight :", weight_path)
assert weight_path and os.path.isfile(weight_path), "시작 가중치를 찾지 못했습니다."

# --- 2) 유틸 ------------------------------------------------
def reset_runtime():
    gc.collect()
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms(False)

def latest_weight(run_dir):
    """run_dir/weights 안에서 best→last 우선 반환"""
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    return None

# --- 3) 파인튜닝 3-스테이지(각 스테이지마다 새 YOLO 인스턴스) -------------------
# Stage A: Stabilize (60 epochs @640)
reset_runtime()
mA = YOLO(weight_path)  # sweep에서 찾은 시작 가중치
runA = os.path.join(WORK_DIR, "finalA_stabilize")
mA.train(
    data=DATA_YAML, epochs=60, imgsz=640, device=0,
    batch=8, workers=2, project=WORK_DIR, name="finalA_stabilize",
    verbose=True, patience=30, optimizer="auto",
    mosaic=0.10, mixup=0.05, close_mosaic=10,
    fliplr=0.5, flipud=0.05,
    degrees=6.0, translate=0.05, scale=0.15, shear=2.0,
    perspective=0.0, hsv_h=0.015, hsv_s=0.45, hsv_v=0.35,
    dropout=0.10, weight_decay=0.001, label_smoothing=0.05,
    deterministic=True, seed=0, cos_lr=False, lr0=5e-4, lrf=0.01,
)
mA.val(data=DATA_YAML, imgsz=640, workers=2)
wA = latest_weight(runA); assert wA, "Stage A 가중치를 찾지 못했습니다."
print("✔ Stage A weight:", wA)

# Stage B: Refine (40 epochs @768)
reset_runtime()
mB = YOLO(wA)  # 새로 로드
runB = os.path.join(WORK_DIR, "finalB_refine")
mB.train(
    data=DATA_YAML, epochs=40, imgsz=768, device=0,
    batch=8, workers=2, project=WORK_DIR, name="finalB_refine",
    resume=False,
    mosaic=0.05, mixup=0.03, close_mosaic=10,
    fliplr=0.5, flipud=0.0,
    degrees=4.0, translate=0.04, scale=0.12, shear=1.0,
    perspective=0.0, hsv_h=0.015, hsv_s=0.35, hsv_v=0.30,
    dropout=0.10, weight_decay=0.001, label_smoothing=0.03,
    deterministic=True, seed=0, cos_lr=True, lr0=4e-4, lrf=0.01,
)
mB.val(data=DATA_YAML, imgsz=768, workers=2)
wB = latest_weight(runB); assert wB, "Stage B 가중치를 찾지 못했습니다."
print("✔ Stage B weight:", wB)

# Stage C: Polish (50 epochs @768, no aug)
reset_runtime()
mC = YOLO(wB)  # 다시 로드
runC = os.path.join(WORK_DIR, "finalC_polish")
mC.train(
    data=DATA_YAML, epochs=50, imgsz=768, device=0,
    batch=8, workers=2, project=WORK_DIR, name="finalC_polish",
    resume=False,
    mosaic=0.0, mixup=0.0, copy_paste=0.0,
    fliplr=0.0, flipud=0.0,
    degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
    perspective=0.0, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
    dropout=0.05, weight_decay=0.001, label_smoothing=0.0,
    deterministic=True, seed=0, cos_lr=True, lr0=3e-4, lrf=0.01,
)
rC = mC.val(data=DATA_YAML, imgsz=768, workers=2)
print("📊 Stage C mAP@[.5:.95]:", float(rC.box.map))

# --- 4) SWA(최근 체크포인트 평균) ----------------------------------------------
def average_state_dicts(paths):
    import torch
    avg = None; n = 0
    for p in paths:
        sd = torch.load(p, map_location="cpu")
        key = "model" if "model" in sd else ("ema" if "ema" in sd else None)
        if key is None:
            continue
        state = sd[key].float().state_dict() if hasattr(sd[key], "state_dict") else sd[key]
        if avg is None:
            avg = {k: v.clone().float() for k, v in state.items()}
        else:
            for k in avg:
                avg[k] += state[k].float()
        n += 1
    if avg is None or n == 0:
        return None
    for k in avg:
        avg[k] /= n
    return avg

ckpt_dir = os.path.join(WORK_DIR, "finalC_polish", "weights")
ckpts = sorted(glob.glob(os.path.join(ckpt_dir, "epoch*.pt")))
ckpts = ckpts[-5:] if len(ckpts) >= 5 else ckpts

if ckpts:
    swa_sd = average_state_dicts(ckpts)
    if swa_sd is not None:
        swa_path = os.path.join(ckpt_dir, "swa.pt")
        model_for_swa = mC.model
        model_for_swa.load_state_dict(swa_sd, strict=False)
        model_for_swa.save(swa_path)
        print(f"🌀 SWA weight saved: {swa_path}")

# --- 5) 최종 검증(best/last/swa 비교) + 승자 선택 ------------------------------
def safe_val(weight_path, imgsz):
    try:
        mm = YOLO(weight_path)
        return float(mm.val(data=DATA_YAML, imgsz=imgsz, workers=2).box.map)
    except Exception as e:
        print("val error:", e)
        return -1.0

final_best = os.path.join(ckpt_dir, "best.pt")
final_last = os.path.join(ckpt_dir, "last.pt")
final_swa  = os.path.join(ckpt_dir, "swa.pt")

scores = {
    "best": safe_val(final_best, 768) if os.path.isfile(final_best) else -1,
    "last": safe_val(final_last, 768) if os.path.isfile(final_last) else -1,
    "swa" : safe_val(final_swa,  768) if os.path.isfile(final_swa)  else -1,
}
print("🔎 Final scores (mAP@[.5:.95] @768):", scores)

paths = {"best": final_best, "last": final_last, "swa": final_swa}
valid = {k: v for k, v in scores.items() if v is not None and v >= 0 and os.path.isfile(paths.get(k, ""))}
assert len(valid) > 0, "유효한 최종 가중치가 없습니다."

winner = max(valid, key=valid.get)
best_model_path = paths[winner]
print(f"🏁 Winner: {winner} ({valid[winner]:.4f})")
print(f"🏁 FINAL_BEST_PT = {best_model_path}")

# 요약 저장
summary = {
    "scores": scores,
    "winner": {"tag": winner, "map": valid[winner], "path": best_model_path},
    "data_yaml": DATA_YAML,
    "work_dir": WORK_DIR
}
with open(os.path.join(WORK_DIR, "final_summary.json"), "w", encoding="utf-8") as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)

# 전역 변수로 노출 (추론/후처리 셀에서 바로 사용)
globals()["FINAL_BEST_PT"] = best_model_path

WORK_DIR : /content/work_yolov8 | exists: True
DATA_YAML: /content/work_yolov8/yolo_dataset/data_abs.yaml | exists: True
🏆 sweep best model tag: yolov8m.pt
🏁 Start weight : /content/work_yolov8/sweep_yolov8m/weights/best.pt
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/work_yolov8/yolo_dataset/data_abs.yaml, degrees=6.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.1, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.05, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.45, hsv_v=0.35, imgsz=640, int8=False, iou=0.

In [ ]:
# =========================================================
# Intentional Overfit 4-Pass Pipeline (Safe Fuse) + TTA → CSV
# =========================================================
import os, glob, gc, json, time, random
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image

# ===== 0) 경로/입력 기본값 =====
WORK_DIR   = os.path.abspath("./work_yolov8")
YOLO_DIR   = os.path.join(WORK_DIR, "yolo_dataset")
DATA_YAML  = os.path.join(YOLO_DIR, "data_abs.yaml")
TEST_DIR   = os.path.abspath("./data/ai05-level1-project/test_images")  # 필요시 수정

os.makedirs(WORK_DIR, exist_ok=True)
assert os.path.isfile(DATA_YAML), f"[DATA_YAML 없음] {DATA_YAML}"
assert os.path.isdir(TEST_DIR),   f"[TEST_DIR 없음] {TEST_DIR}"

# ===== 1) sweep 결과에서 base 모델 태그 확정 =====
CSV_PATH = os.path.join(WORK_DIR, "sweep_results_imgsz640_resilient.csv")
assert os.path.isfile(CSV_PATH), f"스윕 결과 CSV가 없습니다: {CSV_PATH}"

df = pd.read_csv(CSV_PATH)
assert len(df) > 0, "스윕 결과가 비어있습니다."
best_row = df.sort_values("mAP@[.5:.95]", ascending=False).iloc[0]
best_model_name = str(best_row["model"]).strip()  # ex) 'yolov8s.pt' | 'yolov8m.pt' | 'yolov8l.pt'

# 사용자가 강제 지정하고 싶다면 여기서 교체
BASE_MODEL_TAG = best_model_name
print(f"🏆 sweep best model tag: {best_model_name}")
print(f"🎯 BASE_MODEL_TAG      : {BASE_MODEL_TAG}")

# ===== 2) 런타임 유틸 =====
def reset_runtime():
    gc.collect()
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = False
        torch.use_deterministic_algorithms(False)
    except Exception:
        pass

def latest_weight(run_dir):
    """run_dir/weights 안에서 best→last 우선 반환"""
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    # epoch*.pt 중 최신
    cands = sorted(glob.glob(os.path.join(run_dir, "weights", "epoch*.pt")))
    return cands[-1] if cands else None

# ===== 3) cat id 매핑(필요 시) =====
def ensure_idx_to_catid(DATA_YAML, YOLO_DIR):
    """
    idx -> category_id 매핑을 리턴.
    우선순위:
      1) YOLO_DIR/catid_to_idx.json 존재 시 역매핑
      2) DATA_YAML의 names가 숫자 리스트면 category_id로 사용
      3) fallback: idx 자체를 category_id로 사용
    """
    mapping_json = os.path.join(YOLO_DIR, "catid_to_idx.json")
    if os.path.isfile(mapping_json):
        with open(mapping_json, "r", encoding="utf-8") as f:
            catid_to_idx = json.load(f)  # {cat_id: idx}
        return {int(v): int(k) for k, v in catid_to_idx.items()}

    import yaml
    with open(DATA_YAML, "r", encoding="utf-8") as f:
        y = yaml.safe_load(f)
    names = y.get("names", [])
    try:
        nums = [int(x) for x in names]
        if len(nums) == len(names):
            return {i: nums[i] for i in range(len(nums))}
    except Exception:
        pass
    nc = y.get("nc", len(names))
    return {i: i for i in range(nc)}

IDX_TO_CATID = ensure_idx_to_catid(DATA_YAML, YOLO_DIR)
# --- 교체용: 안전 Fuse (템플릿을 ckpt에서 생성) ---
def fuse_checkpoints_safely(paths, out_path, template_ckpt=None):
    """
    paths: 평균낼 ckpt 경로 리스트 (예: [wA, wB])
    template_ckpt: 출력 모델의 구조(헤드 포함)를 제공할 기준 ckpt (예: wA)
    """
    from ultralytics import YOLO
    import torch

    assert len(paths) > 0, "fuse 대상이 비었습니다."
    if template_ckpt is None:
        template_ckpt = paths[0]

    # 1) 템플릿 모델(헤드 모양 포함)을 ckpt에서 생성
    template_model = YOLO(template_ckpt).model  # ← 헤드 shape가 데이터셋 nc에 맞음

    # 2) 각 ckpt에서 state_dict만 추출
    state_dicts = []
    for p in paths:
        if not os.path.isfile(p):
            print(f"skip (missing): {p}")
            continue
        try:
            sd = YOLO(p).model.state_dict()
            state_dicts.append(sd)
            print(f"✔ fuse candidate OK: {p}")
        except Exception as e:
            print(f"skip (load err): {p} ({e})")

    assert len(state_dicts) > 0, "유효한 state_dict가 없습니다."

    # 3) 공통 key만 평균(모양이 같은 key만)
    common_keys = set(state_dicts[0].keys())
    for sd in state_dicts[1:]:
        common_keys &= set(sd.keys())

    avg = {}
    for k in common_keys:
        # 모양이 다르면 제외
        shapes = {tuple(sd[k].shape) for sd in state_dicts}
        if len(shapes) != 1:
            continue
        avg[k] = sum(sd[k] for sd in state_dicts) / len(state_dicts)

    # 4) 템플릿에 로드(strict=False로 누락/불일치 무시)
    missing, unexpected = template_model.load_state_dict(avg, strict=False)
    # (참고 출력) 누락/예상외 키는 대부분 EMA/옵티마/스케줄러 등 부수키거나 모양불일치 헤드 용
    if missing or unexpected:
        print(f"[Fuse] missing: {len(missing)}, unexpected: {len(unexpected)}")

    # 5) 저장
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    template_model.save(out_path)
    print(f"🌀 fused weights saved: {out_path}")
    return out_path

# ===== 5) 4-Pass 설정 =====
from ultralytics import YOLO

DEVICE       = 0
NUM_WORKERS  = 2
IMGSZ_A      = 640   # P1
IMGSZ_B      = 768   # P2
IMGSZ_C      = 768   # P3
IMGSZ_D      = 768   # P4

# ===== 6) P1(base) =====
reset_runtime()
runA = os.path.join(WORK_DIR, "P1_base")
mA = YOLO(BASE_MODEL_TAG)
mA.train(
    data=DATA_YAML, imgsz=IMGSZ_A, epochs=40, device=DEVICE,
    batch=16, workers=NUM_WORKERS, project=WORK_DIR, name="P1_base",
    patience=20, verbose=True,
    mosaic=0.10, mixup=0.05, close_mosaic=10,
    fliplr=0.5, flipud=0.05,
    degrees=5.0, translate=0.08, scale=0.20, shear=1.0,
    hsv_h=0.015, hsv_s=0.45, hsv_v=0.35,
    cos_lr=False, lr0=8e-4, lrf=0.01,
)
mA.val(data=DATA_YAML, imgsz=IMGSZ_A, workers=NUM_WORKERS)
wA = latest_weight(runA); assert wA, "P1 가중치 없음"
print("✔ P1 weight:", wA)

# ===== 7) P2(tune) — P1 이어서 과적합 성향 강화 =====
reset_runtime()
runB = os.path.join(WORK_DIR, "P2_tune")
mB = YOLO(wA)
mB.train(
    data=DATA_YAML, imgsz=IMGSZ_B, epochs=50, device=DEVICE,
    batch=16, workers=NUM_WORKERS, project=WORK_DIR, name="P2_tune",
    patience=25, verbose=True,
    mosaic=0.05, mixup=0.03, close_mosaic=10,
    fliplr=0.5, flipud=0.0,
    degrees=3.0, translate=0.04, scale=0.12, shear=0.5,
    hsv_h=0.010, hsv_s=0.35, hsv_v=0.30,
    cos_lr=True, lr0=6e-4, lrf=0.01,
)
mB.val(data=DATA_YAML, imgsz=IMGSZ_B, workers=NUM_WORKERS)
wB = latest_weight(runB); assert wB, "P2 가중치 없음"
print("✔ P2 weight:", wB)

# ===== 8) P3(fuse) — P1/P2 평균 후 폴리싱 =====
reset_runtime()
# fused_path = os.path.join(WORK_DIR, "weights", "fused_P1P2.pt")
# fused = fuse_checkpoints_safely([wA, wB], fused_path, template_ckpt=wA)

runC = os.path.join(WORK_DIR, "P3_fuse_polish")
mC = YOLO(wB)
mC.train(
    data=DATA_YAML, imgsz=IMGSZ_C, epochs=30, device=DEVICE,
    batch=16, workers=NUM_WORKERS, project=WORK_DIR, name="P3_fuse_polish",
    patience=15, verbose=True,
    mosaic=0.02, mixup=0.00, close_mosaic=10,  # 약하게
    fliplr=0.3, flipud=0.0,
    degrees=2.0, translate=0.02, scale=0.08, shear=0.0,
    hsv_h=0.006, hsv_s=0.25, hsv_v=0.20,
    cos_lr=True, lr0=4e-4, lrf=0.01,
)
mC.val(data=DATA_YAML, imgsz=IMGSZ_C, workers=NUM_WORKERS)
wC = latest_weight(runC); assert wC, "P3 가중치 없음"
print("✔ P3 weight:", wC)

# ===== 9) P4(overfit) — 무증강 + 낮은 LR로 마무리 과적합 =====
reset_runtime()
runD = os.path.join(WORK_DIR, "P4_overfit")
mD = YOLO(wC)
mD.train(
    data=DATA_YAML, imgsz=IMGSZ_D, epochs=30, device=DEVICE,
    batch=16, workers=NUM_WORKERS, project=WORK_DIR, name="P4_overfit",
    patience=10, verbose=True,
    mosaic=0.0, mixup=0.0, copy_paste=0.0,
    fliplr=0.0, flipud=0.0,
    degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
    hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
    cos_lr=True, lr0=3e-4, lrf=0.01,
)
mD.val(data=DATA_YAML, imgsz=IMGSZ_D, workers=NUM_WORKERS)
FINAL_PT = latest_weight(runD); assert FINAL_PT, "P4 가중치 없음"
print("🏁 FINAL_PT:", FINAL_PT)

# ===== 10) TTA 예측 → CSV =====
from ultralytics import YOLO as _YOLO

def predict_to_csv(model_or_path, test_dir, out_csv, imgsz=768,
                   conf=0.20, iou=0.65, max_det=2000, topk=4,
                   tta=True, agnostic=True, idx_to_catid=None):
    model = model_or_path if isinstance(model_or_path, _YOLO) else _YOLO(model_or_path)
    rows, ann_id = [], 1
    image_files = [f for f in os.listdir(test_dir)
                   if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]
    image_files.sort()
    for fn in image_files:
        ip = os.path.join(test_dir, fn)
        r = model.predict(
            source=ip, imgsz=imgsz, conf=conf, iou=iou,
            agnostic_nms=agnostic, max_det=max_det,
            augment=tta, verbose=False
        )[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)

        order = np.argsort(-confs)[:topk]
        for i in order:
            x1, y1, x2, y2 = boxes[i]
            cid = int(idx_to_catid.get(clss[i], clss[i])) if idx_to_catid else int(clss[i])
            rows.append({
                "annotation_id": int(ann_id),
                "image_id": Path(fn).stem,
                "category_id": cid,
                "bbox_x": round(float(x1), 2),
                "bbox_y": round(float(y1), 2),
                "bbox_w": round(float(x2 - x1), 2),
                "bbox_h": round(float(y2 - y1), 2),
                "score": round(float(confs[i]), 6)
            })
            ann_id += 1

    df = pd.DataFrame(rows, columns=[
        "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
    ])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_csv, len(df)

CSV_OUT = os.path.join(WORK_DIR, f"predictions_overfit_{int(time.time())}.csv")
csv_path, nrow = predict_to_csv(FINAL_PT, TEST_DIR, CSV_OUT,
                                imgsz=IMGSZ_D, conf=0.20, iou=0.65, topk=4,
                                tta=True, agnostic=True, idx_to_catid=IDX_TO_CATID)
print(f"✅ CSV saved: {csv_path} (rows={nrow})")

# ===== 11) Colab 다운로드(옵션) =====
try:
    from google.colab import files
    files.download(csv_path)
except Exception:
    pass

🏆 sweep best model tag: yolov8m.pt
🎯 BASE_MODEL_TAG      : yolov8m.pt
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/work_yolov8/yolo_dataset/data_abs.yaml, degrees=5.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.05, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.45, hsv_v=0.35, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0008, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=0.1, multi_scale=False, name=P1_base2, nbs=64, nms=False, opset=None, opt

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =========================================================
# Multi-Split Intentional Overfit 4-Pass (Safe Fuse-ready) + TTA → CSV
# - Splits: R20/R15/R10/R05 등 각자 data_abs.yaml 사용
# - 각 분할별 P1→P2→P3→P4 과적합 학습 후 TTA 예측 CSV 생성
# - run summary CSV까지 저장
# =========================================================
import os, glob, gc, json, time
import numpy as np
import pandas as pd
from pathlib import Path

# ===== 0) 경로/입력 기본값 =====
WORK_DIR   = os.path.abspath("./work_yolov8")
YOLO_ROOT  = os.path.join(WORK_DIR, "yolo_dataset")  # 분할별 YOLO 루트
TEST_DIR   = os.path.abspath("./data/ai05-level1-project/test_images")  # 필요시 수정

# 실험할 분할 태그들 (필요에 맞게 수정)
SPLIT_TAGS = ["R20", "R15", "R10", "R05"]

# 스윕 결과 CSV(베이스 모델 태그 선택에 사용)
CSV_PATH = os.path.join(WORK_DIR, "sweep_results_imgsz640_resilient.csv")

# 4-Pass 공통 하이퍼파라미터(원하면 조절)
DEVICE       = 0
NUM_WORKERS  = 2
IMGSZ_A      = 640   # P1
IMGSZ_B      = 768   # P2
IMGSZ_C      = 768   # P3
IMGSZ_D      = 768   # P4
EPOCHS_A     = 200
EPOCHS_B     = 200
EPOCHS_C     = 200
EPOCHS_D     = 200

assert os.path.isdir(TEST_DIR), f"[TEST_DIR 없음] {TEST_DIR}"
os.makedirs(WORK_DIR, exist_ok=True)

# ===== 1) sweep 결과에서 base 모델 태그 확정 =====
assert os.path.isfile(CSV_PATH), f"스윕 결과 CSV가 없습니다: {CSV_PATH}"
df_sweep = pd.read_csv(CSV_PATH)
assert len(df_sweep) > 0, "스윕 결과가 비어있습니다."
best_row = df_sweep.sort_values("mAP@[.5:.95]", ascending=False).iloc[0]
BASE_MODEL_TAG = str(best_row["model"]).strip()  # ex) 'yolov8s.pt' | 'yolov8m.pt' | 'yolov8l.pt'
print(f"🏆 sweep best model tag: {BASE_MODEL_TAG}")

# ===== 2) 런타임 유틸 =====
def reset_runtime():
    gc.collect()
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
        torch.backends.cudnn.benchmark = False
        torch.use_deterministic_algorithms(False)
    except Exception:
        pass

def latest_weight(run_dir):
    """run_dir/weights 안에서 best→last→epoch*.pt 최신 우선 반환"""
    w_best = os.path.join(run_dir, "weights", "best.pt")
    w_last = os.path.join(run_dir, "weights", "last.pt")
    if os.path.isfile(w_best): return w_best
    if os.path.isfile(w_last): return w_last
    cands = sorted(glob.glob(os.path.join(run_dir, "weights", "epoch*.pt")))
    return cands[-1] if cands else None

def ensure_idx_to_catid(data_yaml_path, yolo_dir_for_split):
    """
    idx -> category_id 매핑을 리턴.
    우선순위:
      1) <split yolo>/catid_to_idx.json 존재 시 역매핑
      2) data.yaml names가 숫자 리스트면 category_id로 사용
      3) fallback: idx 자체를 category_id로 사용
    """
    mapping_json = os.path.join(yolo_dir_for_split, "catid_to_idx.json")
    if os.path.isfile(mapping_json):
        with open(mapping_json, "r", encoding="utf-8") as f:
            catid_to_idx = json.load(f)  # {cat_id: idx}
        return {int(v): int(k) for k, v in catid_to_idx.items()}

    import yaml
    with open(data_yaml_path, "r", encoding="utf-8") as f:
        y = yaml.safe_load(f)
    names = y.get("names", [])
    try:
        nums = [int(x) for x in names]
        if len(nums) == len(names):
            return {i: nums[i] for i in range(len(nums))}
    except Exception:
        pass
    nc = y.get("nc", len(names))
    return {i: i for i in range(nc)}

# (선택) P1/P2 체크포인트 평균 fuse가 필요하면 이 함수 사용 (기본은 P2 그대로 진행)
def fuse_checkpoints_safely(paths, out_path, template_ckpt=None):
    """
    paths: 평균낼 ckpt 경로 리스트 (예: [wA, wB])
    template_ckpt: 출력 모델의 구조(헤드 포함)를 제공할 기준 ckpt (예: wA)
    """
    from ultralytics import YOLO
    import torch

    assert len(paths) > 0, "fuse 대상이 비었습니다."
    if template_ckpt is None:
        template_ckpt = paths[0]

    template_model = YOLO(template_ckpt).model  # 헤드 shape 일치
    state_dicts = []
    for p in paths:
        if not os.path.isfile(p):
            print(f"skip (missing): {p}")
            continue
        try:
            sd = YOLO(p).model.state_dict()
            state_dicts.append(sd)
            print(f"✔ fuse candidate OK: {p}")
        except Exception as e:
            print(f"skip (load err): {p} ({e})")

    assert len(state_dicts) > 0, "유효한 state_dict가 없습니다."

    # 공통 키만 평균(모양 불일치 키는 제외)
    common_keys = set(state_dicts[0].keys())
    for sd in state_dicts[1:]:
        common_keys &= set(sd.keys())

    avg = {}
    for k in common_keys:
        shapes = {tuple(sd[k].shape) for sd in state_dicts}
        if len(shapes) != 1:
            continue
        avg[k] = sum(sd[k] for sd in state_dicts) / len(state_dicts)

    missing, unexpected = template_model.load_state_dict(avg, strict=False)
    if missing or unexpected:
        print(f"[Fuse] missing: {len(missing)}, unexpected: {len(unexpected)}")

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    template_model.save(out_path)
    print(f"🌀 fused weights saved: {out_path}")
    return out_path

# ===== 3) 예측 → CSV =====
from ultralytics import YOLO as _YOLO

def predict_to_csv(model_or_path, test_dir, out_csv, imgsz=768,
                   conf=0.20, iou=0.65, max_det=2000, topk=4,
                   tta=True, agnostic=True, idx_to_catid=None):
    model = model_or_path if isinstance(model_or_path, _YOLO) else _YOLO(model_or_path)
    rows, ann_id = [], 1
    image_files = [f for f in os.listdir(test_dir)
                   if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))]
    image_files.sort()
    for fn in image_files:
        ip = os.path.join(test_dir, fn)
        r = model.predict(
            source=ip, imgsz=imgsz, conf=conf, iou=iou,
            agnostic_nms=agnostic, max_det=max_det,
            augment=tta, verbose=False
        )[0]
        if r.boxes is None or len(r.boxes) == 0:
            continue
        boxes = r.boxes.xyxy.cpu().numpy()
        confs = r.boxes.conf.cpu().numpy()
        clss  = r.boxes.cls.cpu().numpy().astype(int)

        order = np.argsort(-confs)[:topk]
        for i in order:
            x1, y1, x2, y2 = boxes[i]
            cid = int(idx_to_catid.get(clss[i], clss[i])) if idx_to_catid else int(clss[i])
            rows.append({
                "annotation_id": int(ann_id),
                "image_id": Path(fn).stem,
                "category_id": cid,
                "bbox_x": round(float(x1), 2),
                "bbox_y": round(float(y1), 2),
                "bbox_w": round(float(x2 - x1), 2),
                "bbox_h": round(float(y2 - y1), 2),
                "score": round(float(confs[i]), 6)
            })
            ann_id += 1

    df = pd.DataFrame(rows, columns=[
        "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
    ])
    df.to_csv(out_csv, index=False, encoding="utf-8-sig")
    return out_csv, len(df)

# ===== 4) 단일 split을 4-Pass로 실행하는 함수 =====
from ultralytics import YOLO

def run_four_pass_for_split(tag: str):
    yolo_dir = os.path.join(YOLO_ROOT, tag)
    data_yaml = os.path.join(yolo_dir, "data_abs.yaml")
    assert os.path.isfile(data_yaml), f"[{tag}] data_abs.yaml 없음: {data_yaml}"
    assert os.path.isdir(yolo_dir),  f"[{tag}] YOLO 디렉토리 없음: {yolo_dir}"

    idx_to_catid = ensure_idx_to_catid(data_yaml, yolo_dir)

    # ----- P1 -----
    reset_runtime()
    runA = os.path.join(WORK_DIR, f"{tag}_P1_base")
    mA = YOLO(BASE_MODEL_TAG)
    mA.train(
        data=data_yaml, imgsz=IMGSZ_A, epochs=EPOCHS_A, device=DEVICE,
        batch=8, workers=NUM_WORKERS, project=WORK_DIR, name=f"{tag}_P1_base",
        patience=999, verbose=True,
        mosaic=0.10, mixup=0.05, close_mosaic=10,
        fliplr=0.5, flipud=0.05,
        degrees=5.0, translate=0.08, scale=0.20, shear=1.0,
        hsv_h=0.015, hsv_s=0.45, hsv_v=0.35,
        cos_lr=False, lr0=8e-4, lrf=0.01,
    )
    mA.val(data=data_yaml, imgsz=IMGSZ_A, workers=NUM_WORKERS)
    wA = latest_weight(runA); assert wA, f"[{tag}] P1 가중치 없음"

    # ----- P2 -----
    reset_runtime()
    runB = os.path.join(WORK_DIR, f"{tag}_P2_tune")
    mB = YOLO(wA)
    mB.train(
        data=data_yaml, imgsz=IMGSZ_B, epochs=EPOCHS_B, device=DEVICE,
        batch=8, workers=NUM_WORKERS, project=WORK_DIR, name=f"{tag}_P2_tune",
        patience=999, verbose=True,
        mosaic=0.05, mixup=0.03, close_mosaic=10,
        fliplr=0.5, flipud=0.0,
        degrees=3.0, translate=0.04, scale=0.12, shear=0.5,
        hsv_h=0.010, hsv_s=0.35, hsv_v=0.30,
        cos_lr=True, lr0=6e-4, lrf=0.01,
    )
    mB.val(data=data_yaml, imgsz=IMGSZ_B, workers=NUM_WORKERS)
    wB = latest_weight(runB); assert wB, f"[{tag}] P2 가중치 없음"

    # ----- P3 (기본: P2 이어서 폴리싱; fuse 쓰고 싶으면 주석 해제) -----
    reset_runtime()
    runC = os.path.join(WORK_DIR, f"{tag}_P3_polish")
    # # fuse 사용할 경우:
    # fused_path = os.path.join(WORK_DIR, "weights", f"{tag}_fused_P1P2.pt")
    # fuse_checkpoints_safely([wA, wB], fused_path, template_ckpt=wA)
    # mC = YOLO(fused_path)

    # fuse 없이 P2에서 이어서:
    mC = YOLO(wB)
    mC.train(
        data=data_yaml, imgsz=IMGSZ_C, epochs=EPOCHS_C, device=DEVICE,
        batch=8, workers=NUM_WORKERS, project=WORK_DIR, name=f"{tag}_P3_polish",
        patience=999, verbose=True,
        mosaic=0.02, mixup=0.00, close_mosaic=10,
        fliplr=0.3, flipud=0.0,
        degrees=2.0, translate=0.02, scale=0.08, shear=0.0,
        hsv_h=0.006, hsv_s=0.25, hsv_v=0.20,
        cos_lr=True, lr0=4e-4, lrf=0.01,
    )
    mC.val(data=data_yaml, imgsz=IMGSZ_C, workers=NUM_WORKERS)
    wC = latest_weight(runC); assert wC, f"[{tag}] P3 가중치 없음"

    # ----- P4 (Overfit) -----
    reset_runtime()
    runD = os.path.join(WORK_DIR, f"{tag}_P4_overfit")
    mD = YOLO(wC)
    mD.train(
        data=data_yaml, imgsz=IMGSZ_D, epochs=EPOCHS_D, device=DEVICE,
        batch=8, workers=NUM_WORKERS, project=WORK_DIR, name=f"{tag}_P4_overfit",
        patience=999, verbose=True,
        mosaic=0.0, mixup=0.0, copy_paste=0.0,
        fliplr=0.0, flipud=0.0,
        degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
        hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
        cos_lr=True, lr0=3e-4, lrf=0.01,
    )
    mD_val = mD.val(data=data_yaml, imgsz=IMGSZ_D, workers=NUM_WORKERS)
    FINAL_PT = latest_weight(runD); assert FINAL_PT, f"[{tag}] P4 가중치 없음"

    # ----- 예측 CSV -----
    timestamp = int(time.time())
    csv_out = os.path.join(WORK_DIR, f"predictions_{tag}_overfit_{timestamp}.csv")
    predict_to_csv(FINAL_PT, TEST_DIR, csv_out,
                   imgsz=IMGSZ_D, conf=0.20, iou=0.65, topk=4,
                   tta=True, agnostic=True, idx_to_catid=idx_to_catid)

    # 결과 리턴(요약용)
    return {
        "tag": tag,
        "data_yaml": data_yaml,
        "final_weight": FINAL_PT,
        "csv": csv_out,
        "map@[.5:.95]@P4": float(mD_val.box.map) if hasattr(mD_val, "box") else None,
        "map@50@P4": float(mD_val.box.map50) if hasattr(mD_val, "box") else None,
        "map@75@P4": float(mD_val.box.map75) if hasattr(mD_val, "box") else None,
    }

# ===== 5) 모든 분할 실행 & 요약 저장 =====
all_rows = []
for tag in SPLIT_TAGS:
    print(f"\n==================== [RUN SPLIT: {tag}] ====================")
    row = run_four_pass_for_split(tag)
    print(f"✅ [{tag}] DONE | FINAL_PT: {row['final_weight']} | CSV: {row['csv']}")
    all_rows.append(row)

summary_df = pd.DataFrame(all_rows)
summary_csv = os.path.join(WORK_DIR, "overfit_runs_summary.csv")
summary_df.to_csv(summary_csv, index=False, encoding="utf-8-sig")
print("\n📊 Summary saved:", summary_csv)

# (선택) Colab 다운로드
try:
    from google.colab import files
    files.download(summary_csv)
except Exception:
    pass

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
             카나브정 60mg          6          6      0.969          1      0.995      0.995
             울트라셋이알서방정          5          5      0.959          1      0.995      0.978
           졸로푸트정 100mg          2          2      0.965          1      0.995      0.925
          트라젠타정(리나글립틴)          5          5      0.804          1      0.928      0.928
         비모보정 500/20mg          6          6      0.964          1      0.995      0.995
                  레일라정          5          5      0.959          1      0.995      0.995
              리바로정 4mg         13         13      0.915          1      0.995      0.995
            렉사프로정 15mg          2          2          1      0.609      0.995      0.871
     트라젠타듀오정 2.5/850mg          4          4       0.96          1      0.995      0.995
         낙소졸정 500/20mg          4          4      0.949          1      0.995      0.972
       아질렉트정(라사길린메실산염)          4          4      0.949          1      0.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- 4개를 증강다르게 총 에폭이 100번이었는데, 데이터 600개로 학습했을때 0.94
- 이번에 각 에폭마다 200개씩 돌려볼꼐여

- v2 각 트레인 비율을 다르게 하고, 학습시작

### 과거 고정밀추론

In [ ]:
# =========================================================
# 9) 고정밀 추론 (TTA, agnostic NMS)
# =========================================================
best.predict(
    source=os.path.join(YOLO_DIR,"images/val"),
    imgsz=IMGSZ_FINAL,
    conf=0.001, iou=0.7, max_det=2000,
    tta=True, agnostic_nms=True
)

### 고정밀 추론

In [ ]:
# =========================================================
# High-Quality Inference Pipeline (YOLOv8)
# - Single HQ inference (TTA)
# - Multi-scale inference
# - (optional) Ensemble across swa/best/last with WBF merging
# =========================================================
import os, glob, json
from pathlib import Path
from ultralytics import YOLO

# ---------- User Config ----------
# 필수: 작업 폴더 / YOLO 데이터셋 위치
WORK_DIR = globals().get("WORK_DIR", "./work_yolov8")
YOLO_DIR = os.path.join(WORK_DIR, "yolo_dataset")

# 선택: test/val/train 중 자동 선택
TEST_DIR = globals().get("TEST_DIR", os.path.join("./data/ai05-level1-project", "test_images"))

# 고정밀 단일 추론 기본값
IMG_SINGLE = 832     # 768~896 권장
IOU_SINGLE = 0.75
CONF_SINGLE = 0.001
AGNOSTIC_NMS = False
MAX_DET = 3000

# 멀티스케일 사이즈 목록 (비우면 skip)
MULTISCALE_SIZES = [640, 768, 896]  # [] 로 비활성화 가능

# 앙상블 후보 가중치 폴더 (3-Stage 파이널을 썼다면 아래 경로에 있음)
FINAL_WEIGHTS_DIR = os.path.join(WORK_DIR, "finalC_polish", "weights")
ENSEMBLE_USE_WBF = True  # True면 ensemble-boxes로 병합 시도(설치 필요)
WBF_IOU_THR = 0.60
WBF_SKIP_THR = 0.001
WBF_WEIGHTS = [2.0, 1.0, 1.0]  # [swa, best, last] 가중치 예시

# ---------------------------------------------------------
# 0) 추론 소스 자동 선택
VAL_IMG_DIR   = os.path.join(YOLO_DIR, "images", "val")
TRAIN_IMG_DIR = os.path.join(YOLO_DIR, "images", "train")

if TEST_DIR and os.path.isdir(TEST_DIR):
    SOURCE_DIR = TEST_DIR
elif os.path.isdir(VAL_IMG_DIR):
    SOURCE_DIR = VAL_IMG_DIR
else:
    SOURCE_DIR = TRAIN_IMG_DIR

print("🔎 predict source:", SOURCE_DIR)

# ---------------------------------------------------------
# 1) 베스트 가중치 찾기 (우선순위: 파이널 → 스윕 → 최신 best.pt)
def find_best_weight():
    # 파이널 폴더 우선
    for p in [os.path.join(FINAL_WEIGHTS_DIR, "swa.pt"),
              os.path.join(FINAL_WEIGHTS_DIR, "best.pt"),
              os.path.join(FINAL_WEIGHTS_DIR, "last.pt")]:
        if os.path.isfile(p):
            return p

    # 스윕 결과 기반 (BEST_MODEL 있는 경우)
    if "BEST_MODEL" in globals():
        exp_name = f"sweep_{Path(BEST_MODEL).stem}"
        for p in [os.path.join(WORK_DIR, exp_name, "weights", "best.pt"),
                  os.path.join(WORK_DIR, "runs", "detect", exp_name, "weights", "best.pt")]:
            if os.path.isfile(p):
                return p

    # 아니면 work 디렉토리 아래 최신 best.pt
    cands = glob.glob(os.path.join(WORK_DIR, "**", "weights", "best.pt"), recursive=True)
    if cands:
        cands.sort(key=os.path.getmtime, reverse=True)
        return cands[0]
    raise FileNotFoundError("best/swa/last 가중치를 찾지 못했습니다. 학습 셀을 먼저 실행하세요.")

best_weight = find_best_weight()
print("✅ BEST_WEIGHT:", best_weight)

🔎 predict source: ./data/ai05-level1-project/test_images
✅ BEST_WEIGHT: /content/work_yolov8/finalC_polish/weights/best.pt


In [ ]:
# =========================================================
# 베스트 모델 로드 → 고정밀 추론(augment= True, agnostic_nms, max_det↑)
# =========================================================
import os, glob
from pathlib import Path
from ultralytics import YOLO

IMGSZ_FINAL = 640  # 고정

# # 1) BEST_MODEL에서 best.pt 찾기
# assert 'BEST_MODEL' in globals(), "BEST_MODEL 변수가 없습니다. 스윕 셀을 먼저 실행하세요."
# exp_name = f"sweep_{Path(BEST_MODEL).stem}"

# cand_paths = [
#     os.path.join(WORK_DIR, exp_name, "weights", "best.pt"),
#     os.path.join(WORK_DIR, "runs", "detect", exp_name, "weights", "best.pt"),
# ]
# best_weight = None
# for p in cand_paths:
#     if os.path.isfile(p):
#         best_weight = p
#         break
# if best_weight is None:
#     cands = glob.glob(os.path.join(WORK_DIR, "**", "weights", "best.pt"), recursive=True)
#     if not cands:
#         raise FileNotFoundError("best.pt를 찾지 못했습니다. 스윕 셀을 먼저 성공적으로 실행해주세요.")
#     cands.sort(key=os.path.getmtime, reverse=True)
#     best_weight = cands[0]
print("✅ BEST_WEIGHT:", best_weight)

# 2) 모델 로드
best = YOLO(best_weight)

# 3) 추론 소스 선택
VAL_IMG_DIR   = os.path.join(YOLO_DIR, "images", "val")
TRAIN_IMG_DIR = os.path.join(YOLO_DIR, "images", "train")
if 'TEST_DIR' in globals() and os.path.isdir(TEST_DIR):
    SOURCE_DIR = TEST_DIR
elif os.path.isdir(VAL_IMG_DIR):
    SOURCE_DIR = VAL_IMG_DIR
else:
    SOURCE_DIR = TRAIN_IMG_DIR
print("🔎 predict source:", SOURCE_DIR)

# 4) 고정밀 추론 실행 (Ultralytics 8.3.x: tta → augment)
run_name = f"predict_{Path(best_weight).parent.parent.name}_hires{IMGSZ_FINAL}_aug"
pred = best.predict(
    source=SOURCE_DIR,
    imgsz=IMGSZ_FINAL,
    conf=0.001,
    iou=0.70,
    max_det=2000,
    augment=True,        # ← 여기! (tta=True 아님)
    agnostic_nms=True,
    save=True,
    project=WORK_DIR,
    name=run_name
)

print("✅ 고정밀 추론 완료.")
print("📁 결과 폴더:", os.path.join(WORK_DIR, run_name))

✅ BEST_WEIGHT: /content/work_yolov8/finalC_polish/weights/best.pt
🔎 predict source: ./data/ai05-level1-project/test_images

image 1/843 /content/data/ai05-level1-project/test_images/1.png: 640x512 1 보령부스파정 5mg, 2 놀텍정 10mgs, 1 트윈스타정 40/5mg, 1 울트라셋이알서방정, 224.4ms
image 2/843 /content/data/ai05-level1-project/test_images/10.png: 640x512 2 보령부스파정 5mgs, 1 가바토파정 100mg, 1 라비에트정 20mg, 1 레일라정, 28.8ms
image 3/843 /content/data/ai05-level1-project/test_images/100.png: 640x512 1 보령부스파정 5mg, 1 가바토파정 100mg, 1 란스톤엘에프디티정 30mg, 1 신바로정, 28.5ms
image 4/843 /content/data/ai05-level1-project/test_images/1003.png: 640x512 2 기넥신에프정(은행엽엑스)(수출용)s, 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 31.1ms
image 5/843 /content/data/ai05-level1-project/test_images/1004.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000mg, 28.6ms
image 6/843 /content/data/ai05-level1-project/test_images/1005.png: 640x512 1 기넥신에프정(은행엽엑스)(수출용), 1 리피토정 20mg, 1 스토가정 10mg, 1 트윈스타정 40/5mg, 1 제미메트서방정 50/1000m

In [ ]:
# =========================================================
# 5) 베스트 모델 아키텍처로 150 epoch 파이널 학습 (Fine-tuning)
# =========================================================
import gc, torch
from ultralytics import YOLO
from pathlib import Path

IMGSZ_FINAL = 640
EPOCHS_FINAL = 150
BATCH_FINAL = 16

# --- 1) 베스트 모델 구조 기반 새 학습 세션 시작 ---
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()

final_name = f"final_{Path(BEST_MODEL).stem}"

print(f"🚀 Final Training Start: {final_name} (epochs={EPOCHS_FINAL}) {best_weight}")

# model_final = YOLO(best_weight)  # ← architecture만 불러옴 (.pt)

best.train(
    data=DATA_YAML,
    epochs=EPOCHS_FINAL,
    imgsz=IMGSZ_FINAL,
    batch=BATCH_FINAL,
    device=0,
    name=final_name,
    project=WORK_DIR,
    workers=8,
    patience=30,
    verbose=True,
    deterministic=True,
    save=True,   # ✅ 반드시 명시해야 best.pt / last.pt 생성됨
    # 하이퍼파라미터는 그대로 유지 or 필요시 조정
    mosaic=0.0, mixup=0.0, copy_paste=0.0,
    fliplr=0.0, flipud=0.0,
    degrees=0.0, translate=0.0, scale=0.0, shear=0.0,
    perspective=0.0, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0,
    dropout=0.05, weight_decay=0.001, label_smoothing=0.0,
    seed=0, cos_lr=True, lr0=3e-4, lrf=0.01,
)

print("✅ Final training complete.")

# --- 2) 검증 및 저장 ---
r_final = best.val(imgsz=IMGSZ_FINAL, data=DATA_YAML)
print(f"\n📊 Final Validation — mAP@[.5:.95]: {r_final.box.map:.4f} | mAP@.50: {r_final.box.map50:.4f}")

🚀 Final Training Start: final_yolov8m (epochs=150) /content/work_yolov8/finalC_polish/weights/best.pt
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
Ultralytics 8.3.221 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/work_yolov8/yolo_dataset/data_abs.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.05, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.0, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.0, hsv_s=0.0, hsv_v=0.0, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0003, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/con

In [ ]:
# =========================================================
# 10) CSV 저장 (카테고리 ID만, TTA 대응)
# =========================================================
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
from google.colab import files

CSV_OUT = os.path.join(WORK_DIR, "predictions_yolo.csv")
rows = []
annotation_id = 1

# 최신 Ultralytics 버전에서는 predict() 대신 model.predict() 또는 model.tta_predict() 사용
for fn in tqdm(os.listdir(TEST_DIR), desc="Predict (TTA)"):
    if not fn.lower().endswith((".jpg", ".png")):
        continue
    p = os.path.join(TEST_DIR, fn)

    # ✅ 안전한 방식 — TTA 적용 시
    try:
        # Ultralytics >=8.3.50 : TTA는 .predict() + augment=True 로 대체
        r = best.predict(
            source=p,
            imgsz=IMGSZ_FINAL,
            conf=0.3,
            iou=0.6,
            agnostic_nms=True,
            augment=True,     # ✅ tta=True → augment=True 로 변경
            verbose=False
        )[0]
    except TypeError:
        # 하위 버전 호환용 fallback
        r = best(p, imgsz=IMGSZ_FINAL, conf=0.3, iou=0.6, agnostic_nms=True)[0]

    if r.boxes is None:
        continue

    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clss  = r.boxes.cls.cpu().numpy().astype(int)

    # 상위 4개만 남기기
    for i in np.argsort(-confs)[:4]:
        x1, y1, x2, y2 = boxes[i]
        rows.append({
            "annotation_id": annotation_id,
            "image_id": Path(fn).stem,
            "category_id": int(list(catid_to_idx.keys())[clss[i]]),
            "bbox_x": round(float(x1), 2),
            "bbox_y": round(float(y1), 2),
            "bbox_w": round(float(x2 - x1), 2),
            "bbox_h": round(float(y2 - y1), 2),
            "score": round(float(confs[i]), 6)
        })
        annotation_id += 1

# ✅ CSV 저장
df = pd.DataFrame(rows)
df.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")
print("✅ CSV saved:", CSV_OUT)
files.download(CSV_OUT)


Predict (TTA): 100%|██████████| 843/843 [01:03<00:00, 13.27it/s]

✅ CSV saved: /content/work_yolov8/predictions_yolo.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# =========================================================
# 11) EasyOCR 융합 후처리 (신뢰도 보정)
# =========================================================
import easyocr
reader = easyocr.Reader(['ko','en'])

def ocr_boost_on_roi(img, boxes, boost=1.1):
    im = Image.open(img).convert("RGB")
    boosts = np.ones(len(boxes))
    for i,(x1,y1,x2,y2) in enumerate(boxes):
        crop = im.crop((x1,y1,x2,y2))
        res = reader.readtext(np.array(crop))
        joined = " ".join([r[1] for r in res]).lower()
        if any(k in joined for k in ["mg","tab","capsule"]) or any(ch.isdigit() for ch in joined):
            boosts[i]=boost
    return boosts

CSV_OCR = os.path.join(WORK_DIR,"predictions_yolo_ocr.csv")
rows=[]
annotation_id=1
for fn in tqdm(os.listdir(TEST_DIR),desc="OCR"):
    if not fn.lower().endswith((".jpg",".png")): continue
    p=os.path.join(TEST_DIR,fn)
    r=best(p,imgsz=IMGSZ_FINAL,conf=0.25,iou=0.65,tta=True,agnostic_nms=True)[0]
    if r.boxes is None: continue
    boxes=r.boxes.xyxy.cpu().numpy(); confs=r.boxes.conf.cpu().numpy(); clss=r.boxes.cls.cpu().numpy().astype(int)
    boosts=ocr_boost_on_roi(p,boxes)
    confs*=boosts
    for i in np.argsort(-confs)[:4]:
        x1,y1,x2,y2=boxes[i]
        rows.append({
            "annotation_id":annotation_id,"image_id":Path(fn).stem,
            "category_id":int(list(catid_to_idx.keys())[clss[i]]),
            "bbox_x":round(float(x1),2),"bbox_y":round(float(y1),2),
            "bbox_w":round(float(x2-x1),2),"bbox_h":round(float(y2-y1),2),
            "score":round(float(confs[i]),6)
        })
        annotation_id+=1

df = pd.DataFrame(rows)
df.to_csv(CSV_OUT,index=False,encoding="utf-8-sig")
print("✅ CSV saved:", CSV_OUT)

In [ ]:
!find ./work_yolov8 -type f -name "best.pt"

./work_yolov8/sweep_yolov8m/weights/best.pt
./work_yolov8/sweep_yolov8l/weights/best.pt
./work_yolov8/sweep_yolov8s3/weights/best.pt
./work_yolov8/final_yolov8l/weights/best.pt
./work_yolov8/sweep_yolov8s4/weights/best.pt


In [ ]:
# =========================================================
# 파이널 학습 결과물에서 best.pt 경로 수동 지정 (확정)
# =========================================================
best_model_path = "./work_yolov8/sweep_yolov8l/weights/best.pt"
print(f"✅ best_model_path = {best_model_path}")

✅ best_model_path = ./work_yolov8/sweep_yolov8l/weights/best.pt


In [ ]:
# ============================================
# EasyOCR 융합 후처리 → CSV 저장 (category_id만)
# ============================================
import os, math, json, time, gc
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from ultralytics import YOLO
import easyocr
from pycocotools.coco import COCO

# ----------------------------
# 0) 기본 경로/객체 준비
# ----------------------------
# (필요시 재정의)
DATA_ROOT = globals().get("DATA_ROOT", "./data/ai05-level1-project")
WORK_DIR  = globals().get("WORK_DIR", "./work_yolov8")
TRAIN_JSON = globals().get("TRAIN_JSON", f"{DATA_ROOT}/train.json")
TEST_DIR   = globals().get("TEST_DIR",  os.path.join(DATA_ROOT, "test_images"))
IMGSZ_FINAL = globals().get("IMGSZ_FINAL", 640)

BEST_PT = best_model_path


# # 베스트 모델 가중치 경로 (앞 단계에서 만든 결과를 우선 사용)
# if "best_model_path" in globals() and os.path.isfile(best_model_path):
#     BEST_PT = best_model_path
# else:
#     # 혹시 수동 지정 필요하면 아래 경로 수정
#     # 예: BEST_PT = "./work_yolov8/runs/detect/final_yolov8m/weights/best.pt"
#     raise FileNotFoundError("best_model_path가 없습니다. 파이널 학습 셀에서 best pt 경로를 확인해 주세요.")

print("✅ Using BEST model:", BEST_PT)

# COCO category_id ↔ YOLO class index 매핑
coco_train = COCO(TRAIN_JSON)
cat_ids = sorted(coco_train.getCatIds())
catid_to_idx = {cid: i for i, cid in enumerate(cat_ids)}   # COCO id -> YOLO idx
idx_to_catid = {i: cid for cid, i in catid_to_idx.items()} # YOLO idx -> COCO id (우리가 사용할 것)

# ----------------------------
# 1) 모델 & OCR 로더
# ----------------------------
model = best # YOLO(BEST_PT)
# GPU가 있으면 EasyOCR도 GPU 사용 권장
reader = easyocr.Reader(['ko','en'], gpu=True)

# ----------------------------
# 2) OCR 신호 계산 함수
# ----------------------------
KEYWORDS = [
    "mg","tab","tablet","capsule","cap","sr","er","od","cr",
    "정","캡슐","서방정","정제","경질","연질"
]
def ocr_score_from_text(text: str) -> float:
    """
    OCR 텍스트로부터 0~1 사이의 점수를 만듭니다.
    간단히: 숫자 포함 + 키워드 포함 + 길이 보너스
    """
    if not text:
        return 0.0
    t = text.lower()
    has_digit = any(ch.isdigit() for ch in t)
    has_kw = any(k in t for k in KEYWORDS)
    length_bonus = min(len(t) / 10.0, 1.0)  # 10자 이상이면 1.0
    # 가중합 (필요시 조절)
    raw = (0.5 if has_digit else 0.0) + (0.4 if has_kw else 0.0) + (0.2 * length_bonus)
    return float(max(0.0, min(1.0, raw)))  # [0,1]로 클램프

def ocr_boost_for_boxes(img_path, boxes_xyxy):
    """
    각 박스마다 EasyOCR을 돌려 s_ocr 점수를 산출.
    반환: np.ndarray, shape=(N,), 각 요소 ∈ [0,1]
    """
    if len(boxes_xyxy) == 0:
        return np.zeros((0,), dtype=np.float32)

    im = Image.open(img_path).convert("RGB")
    scores = []
    for (x1,y1,x2,y2) in boxes_xyxy:
        # 안전 클램프
        x1, y1 = max(0, int(x1)), max(0, int(y1))
        x2, y2 = max(x1+1, int(x2)), max(y1+1, int(y2))
        crop = im.crop((x1, y1, x2, y2))
        # EasyOCR
        try:
            res = reader.readtext(np.array(crop))
            txt = " ".join([r[1] for r in res])
        except Exception:
            txt = ""
        s = ocr_score_from_text(txt)
        scores.append(s)
    return np.array(scores, dtype=np.float32)

# ----------------------------
# 3) 추론 + OCR 융합
# ----------------------------
CSV_OUT = os.path.join(WORK_DIR, "predictions_yolo_ocr_fused.csv")
MAX_PER_IMAGE = 4
CONF_BASE = 0.25   # YOLO 기본 conf
IOU_NMS  = 0.65    # YOLO NMS
OCR_W    = 0.25    # OCR 가중치 (conf_fused = conf * (1 + OCR_W * s_ocr))

rows = []
annotation_id = 1

# 파일 목록 수집
if not os.path.isdir(TEST_DIR):
    raise FileNotFoundError(f"TEST_DIR가 없습니다: {TEST_DIR}")
test_files = sorted([
    f for f in os.listdir(TEST_DIR)
    if f.lower().endswith((".jpg",".jpeg",".png",".bmp",".webp"))
])

from tqdm import tqdm
for fn in tqdm(test_files, desc="Predict+OCR"):
    ip = os.path.join(TEST_DIR, fn)
    # YOLO 추론 (TTA + agnostic_nms)
    r = model.predict(
        source=ip,
        imgsz=IMGSZ_FINAL,
        conf=CONF_BASE,
        iou=IOU_NMS,
        agnostic_nms=True,
        verbose=False
    )[0]

    if r.boxes is None or r.boxes.xyxy is None or len(r.boxes) == 0:
        continue

    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clss  = r.boxes.cls.cpu().numpy().astype(int)

    # OCR 점수 계산
    s_ocr = ocr_boost_for_boxes(ip, boxes)  # [0,1]
    conf_fused = confs * (1.0 + OCR_W * s_ocr)

    # 상위 4개만
    order = np.argsort(-conf_fused)[:MAX_PER_IMAGE]
    for i in order:
        x1, y1, x2, y2 = boxes[i]
        cat_id = int(idx_to_catid.get(int(clss[i]), -1))  # COCO category_id(숫자) 보존
        rows.append({
            "annotation_id": int(annotation_id),
            "image_id": Path(fn).stem,
            "category_id": cat_id,
            "bbox_x": round(float(x1), 2),
            "bbox_y": round(float(y1), 2),
            "bbox_w": round(float(x2 - x1), 2),
            "bbox_h": round(float(y2 - y1), 2),
            "score": round(float(conf_fused[i]), 6)
        })
        annotation_id += 1

# ----------------------------
# 4) CSV 저장
# ----------------------------
os.makedirs(WORK_DIR, exist_ok=True)
df = pd.DataFrame(rows, columns=[
    "annotation_id","image_id","category_id",
    "bbox_x","bbox_y","bbox_w","bbox_h","score"
])
df.to_csv(CSV_OUT, index=False, encoding="utf-8-sig")
print(f"✅ CSV saved: {CSV_OUT} (rows={len(df)})")

✅ Using BEST model: /content/work_yolov8/finalC_polish/weights/best.pt
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete

Predict+OCR: 100%|██████████| 843/843 [02:12<00:00,  6.35it/s]

✅ CSV saved: /content/work_yolov8/predictions_yolo_ocr_fused.csv (rows=3071)


In [ ]:
# =========================================================
# [시각화] 테스트 이미지에 YOLO 박스 + OCR 텍스트 표시
# =========================================================
import matplotlib.pyplot as plt
import cv2

# 한 번에 표시할 샘플 수 (5개 정도 추천)
N_SHOW = 5

# 표시할 파일 샘플 선택
sample_files = np.random.choice(test_files, size=min(N_SHOW, len(test_files)), replace=False)

for fn in sample_files:
    ip = os.path.join(TEST_DIR, fn)
    img = cv2.imread(ip)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # YOLO 예측
    r = model.predict(
        source=ip,
        imgsz=IMGSZ_FINAL,
        conf=CONF_BASE,
        iou=IOU_NMS,
        agnostic_nms=True,
        verbose=False
    )[0]

    if (r.boxes is None) or (len(r.boxes) == 0):
        print(f"⚠️ No detection in {fn}")
        continue

    boxes = r.boxes.xyxy.cpu().numpy()
    confs = r.boxes.conf.cpu().numpy()
    clss  = r.boxes.cls.cpu().numpy().astype(int)

    # OCR 텍스트 추출
    im_pil = Image.open(ip).convert("RGB")
    for (x1, y1, x2, y2), conf, cls_idx in zip(boxes, confs, clss):
        crop = im_pil.crop((x1, y1, x2, y2))
        res = reader.readtext(np.array(crop))
        txt = " ".join([r[1] for r in res]) if res else ""

        # 박스 색상
        color = (0, 255, 0)
        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        label = f"{idx_to_catid.get(cls_idx, '?')} | {conf:.2f}"
        if txt:
            label += f" | OCR: {txt[:20]}"
        cv2.putText(img, label, (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1, cv2.LINE_AA)

    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.title(fn)
    plt.axis("off")
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# =========================================================
# [성능 평가 시각화] mAP 요약 / IoU별 곡선 / 클래스별 AP Top-N
# 전제: 직전에 `metrics = model.val(imgsz=640)` 을 실행했거나,
#       runs 디렉토리의 results.csv 가 존재함.
# =========================================================
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- 1) metrics 객체에서 기본 지표 뽑기 (가능할 때) ---
maps_iou = None
map_all  = None
map50    = None
map75    = None
per_class_ap = None
class_names = None

try:
    map_all = float(metrics.box.map)      # mAP@[0.5:0.95]
    map50   = float(metrics.box.map50)    # mAP@0.50
    map75   = float(metrics.box.map75)    # mAP@0.75
    # IoU별 mAP (일반적으로 10개 IoU: 0.50~0.95)
    maps_iou = np.array(metrics.box.maps) if hasattr(metrics.box, "maps") else None
    # 클래스별 AP (있으면 사용)
    per_class_ap = getattr(metrics.box, "ap_class", None)
    class_names = getattr(model, "names", None)
except Exception as e:
    print("metrics 객체에서 직접 값 추출 실패:", e)

# --- 2) runs 결과CSV에서 보조적으로 로드 (없으면 스킵) ---
# Ultralytics는 runs/detect/<run_name>/results.csv 로 에폭별 결과를 저장
results_csv = None
try:
    run_dirs = sorted(glob.glob("runs/detect/*"), key=os.path.getmtime)
    if run_dirs:
        latest = run_dirs[-1]
        csv_path = os.path.join(latest, "results.csv")
        if os.path.exists(csv_path):
            results_csv = pd.read_csv(csv_path)
            # 열 후보: 'metrics/mAP50(B)', 'metrics/mAP50-95(B)', 'metrics/precision(B)', 'metrics/recall(B)'
except Exception as e:
    print("results.csv 로드 실패:", e)

# --- 3) 요약 막대 그래프 (mAP, mAP50, mAP75) ---
plt.figure(figsize=(5,4))
vals, labels = [], []
if map_all is not None:
    vals.append(map_all); labels.append("mAP@[0.5:0.95]")
if map50 is not None:
    vals.append(map50);   labels.append("mAP@0.50")
if map75 is not None:
    vals.append(map75);   labels.append("mAP@0.75")

if vals:
    plt.bar(labels, vals)
    for i, v in enumerate(vals):
        plt.text(i, v+0.01, f"{v:.3f}", ha="center", va="bottom", fontsize=10)
    plt.ylim(0, max(vals)+0.1)
    plt.title("Summary mAP")
    plt.ylabel("AP")
    plt.show()
else:
    print("요약 막대 그래프: 표시할 mAP 값이 없습니다 (metrics가 없거나 비어 있음).")

# --- 4) IoU별 mAP 곡선 ---
if maps_iou is not None and maps_iou.size > 0:
    # x축 IoU (Ultralytics 기본 0.50~0.95, 10개)
    iou_thrs = np.linspace(0.50, 0.95, maps_iou.shape[0])
    plt.figure(figsize=(6,4))
    plt.plot(iou_thrs, maps_iou, marker="o")
    plt.title("mAP vs. IoU threshold")
    plt.xlabel("IoU threshold")
    plt.ylabel("mAP")
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("IoU별 mAP 곡선: metrics.box.maps 가 없어 생략합니다.")

# --- 5) 클래스별 AP 바차트 (상위/하위 Top-N) ---
TOP_N = 15  # 필요시 조절
if per_class_ap is not None and class_names is not None:
    # per_class_ap 가 클래스 순서대로 있다고 가정
    ap = np.array(per_class_ap).astype(float)
    idxs = np.arange(len(ap))
    # 이름 준비
    names = [class_names[i] if isinstance(class_names, dict) else str(i) for i in idxs]
    # 상위 TOP_N
    top_idx = np.argsort(-ap)[:min(TOP_N, len(ap))]
    # 하위 TOP_N
    bot_idx = np.argsort(ap)[:min(TOP_N, len(ap))]

    fig, axes = plt.subplots(1,2, figsize=(14,5))
    axes[0].barh([names[i] for i in top_idx][::-1], ap[top_idx][::-1])
    axes[0].set_title(f"Per-class AP Top-{len(top_idx)}")
    axes[0].set_xlabel("AP")

    axes[1].barh([names[i] for i in bot_idx], ap[bot_idx])
    axes[1].set_title(f"Per-class AP Bottom-{len(bot_idx)}")
    axes[1].set_xlabel("AP")

    plt.tight_layout()
    plt.show()
else:
    print("클래스별 AP 그래프: per_class_ap 또는 class_names 가 없어 생략합니다.")

# --- 6) (선택) 에폭별 학습 곡선 결과.csv 요약 ---
if results_csv is not None:
    plt.figure(figsize=(6,4))
    # 컬럼 이름은 버전에 따라 약간 다를 수 있음 → 존재하는 열만 사용
    cols_candidates = [
        ("metrics/mAP50-95(B)", "mAP@[0.5:0.95]"),
        ("metrics/mAP50(B)", "mAP@0.50"),
        ("metrics/precision(B)", "Precision"),
        ("metrics/recall(B)", "Recall")
    ]
    used = 0
    for col, label in cols_candidates:
        if col in results_csv.columns:
            plt.plot(results_csv[col], label=label)
            used += 1
    if used:
        plt.title("Training Curves (per epoch)")
        plt.xlabel("Epoch")
        plt.ylabel("Metric")
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
    else:
        print("results.csv에 plotting 가능한 열이 없습니다. 열 이름을 확인하세요.")
else:
    print("에폭별 학습 곡선: results.csv 를 찾지 못해 생략합니다.")